In [330]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 
import json 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)



In [331]:
# Formatting log
# 6 Dec. 2023
# Going to try to keep better track of changes as I'm doing them so I can add them as remarks to my commit notes
#
# Reformatting, reordering, and correcting system type labels. This involves removing duplicate entries, fixing \
# errors where a slash was missed during the matching process, system types missing altogether, and slimming down \
# the variations. For example, binary quasars --> dual AGNs or candidates, AGN recoil candidate --> \
# recoil candidate, etc., and we're removing things like 'fast jet realignment' and 'double-peak emitter'
# 
# These classes are going to be alphabetized from now on within the System Type cell. 
#
#
# I've noticed I accidently removed the separations for the Hennawi+2010 targets because they were referred to as \
# DelT (and vel diff was DelV). I've added the separations back in. Tytler+2009 is also missing info, but I think \
# I accidently removed it from their table when matching. I also corrected bibcodes --> BibCodes for Tytler+2009 \
# and Kirkman+2008. 
#
# Corrected the missing angular separations for Findlay+2018 (and converted from arcmin to arcsec). Also added \
# in coordinate waveband and source in for (SDSS or ATLAS), equinox, and changed Dual AGN--> Quasar Pair
# We'll remove pairs and keep genuine duals as we format here... Also ajusted the selection and confirmation \
# methods columns. 
#
# Added in the missing separations for Hennawi+2006 (removed them by accident). Correcting the selection and \
# confirmation methods of Hennawi+2006. This includes making adjustments to the t2 table. Separations have also \
# now been added back in for Hennawi+2010. These were being read in as delT_1
#
# In binary quasar notebook, also went back and fixed where I accidently was throwing out the BibCode(s) column
#
# Note for Hennawi's targets, we still have a problem. I was able to remove duplicates before, but I was not \
# careful about paying attention to whether the coordinates and redshifts referred to the same objects, or at least, \
# I have no record of being that careful. I will eventually need to go through by hand for all of the duplicates and \
# ensure the redshifts and coordinates do indeed refer to the same objects. For now, we will ignore this problem. 
#
# I believe I have now also fixed all of the equinox, coordinate waveband, and cordinate source info that was missing \
# from the binary quasar notebook (added back in where ever possible); made '1' and '2' columns for these entries
#
# Corrected the missing column in the Koss+2012 and Hainline+ listing
#
# Modified the selection method for targets from Yang+2019. These are no longer 'strong' and 'probable' X-shaped \
# radio sources; we list them simply as X-shaped radio sources. 
#
# Corrected the equinox, coordinate waveband, and coordinate course listings for X-shaped radio sources. Wrong \
# column names for some, others had wrong info (Yang+2019 for example had coordinates listed as optical and SDSS \
# rather than radio and VLA / FIRST)
#
# In individual table: (1) changed all 'Recoiling AGN Candidate' tags to 'Recoil Candidate'; (2) since X-shaped \
# radio sources have been claimed to be either recoiling AGNs/SMBHs AND binary AGNs, ALL X-shaped radio sources \
# have a 'recoil candidate' and a 'binary AGN candidate' tag. This is also the case in the actual X-shaped radio \
# source notebook; (3) removed use of 'double-peaked emitter candidate' from system types. no need to include.
#
# Appended '1' to all equinox, coordinate waveband, and coordinate source columns in varstrometry notebook. Also
# added in the z1 column (using the z column) in the Orosz table. No z column in the Hwang table, though. I have\
# now corrected issues with the Orosz+ table; RA1 and Dec1 have been added in, and RA2, Dec2, and Name2 have been \
# marked as -99 since no counterparts are currently known. Name2, RA2, Dec2, and the degree versions for the \
# Hwang+ table has also been flagged as -99. Note, the name flags are string(-99). RA1 and Dec1 added as well.
#
# Appended '1' to all equinox, coordinate waveband, and coordinate source columns in binary periodicity notebook.
# Also now added in a z1 column
#
# NEED TO ADD SELECTION METHOD INFO FOR THE MAJORITY OF LENS SELECTED SOURCES (ETC INADA 2008)
#
# In the offset eission line notebook, I've now added in the '1' entries for equinox, waveband, and source
#
# In the double-peaked notebook: (1) added 'candidate' to the object examined by Nandi+; (2) changed equinox, \
# coordinate waveband, and coordinate source columns to have '1' appended at the end; (3) added back in  the\
# redshift 1 column. All redshift 2s were verified to be identical to z1, and thenm overwritten to '-99' along \
# with the spec type and the dV entries. I also now fixed the issue of missing redshifts and coordinates for \
# a good chunk of the Yuan+2016 targets. The z column had been broken into z_x and z_y for some reason, so \
# =yuan2016['z'] obviously didnt work (I guess I coerced the errors away here by accident). Also added in the \
# sexagesimal format for coordinates under RA and Dec, which are then converted to RA1 and Dec1 at the end like \
# usual.
#
# In the offset broad line binary notebook: added in '1' and '2' entries for equinox, waveband, and source, and \
# verified that there are z1 columns. Redshift 2s were verified to be identical to z1s and overwritten by '-99'. \
# Spec types and dVs were also overwritte, except for the dVs from Kim+2016, which were based on Halpha.
#
# In the match all catalogs notebook, I think I fixed the issue where confirmation methods and names were being \
# overwritten by accident for one or two objects. I've also now fixed the issue where a system type flag from one \
# table was beeing added to the MAC without a delimiter.
#
# The dual quasars from lens searches seems okay for now, but I'll be checking for issues tomorrow (today) when \
# I output the latest version of the DR.
#
# Corrected the improper system type adding in the match all catalogs notebook. 
# In binary quasar notebook, changed 'quasar pairs' to quasar pair.
# In individual objects table, changed the Lusso+2019 target from dual quasar to dual agn. This will probably \
# get changed back to something else since this object is likely not a merger induced 'dual'
#
#
# In the lens matching notebook: I have now reformatted the redshift columns for Lemon2020 (z1-z2 because they \
# are NIQs). For cases of missing RA and Decs, I made sure to change it from str(-99) to -99. Reformatted Lemon \
# 2019 and 2018 as well, adjusting the redshifts for 1 and 2 (adding in z1, and making sure z1=z2 for NIQs), and \
# adjusted the string vs nonstring for coordinates of 2. Added in Name1 for Agnello and adjusted the 2 coords to \
# non strings. Agnello already has a redshift 1 column. Adjusted coord for 2 for Spiniello. I've made adjustments \
# to the eftek2017 table; I'd only previously adjusted z1 based on if z1<0, but now I adjust both z1 and 2 based \
# on the -1 flag, and I also modify the z type. I've formatted some of the equinox, waveband, and source entries \
# in more2016. See notes below about the Inada tables; I have done some major reformatting of the Inada csv input \
# files as well as adding for loopd top overwrite bad redshift, z type, and coordinate cells. I've finished adding \
# back in the (largely blank) notes column and finalizing changes to the lens notebook.
#
# It looks like some inconclusive pairs from some of Lemon's works are included (from 2019 I think) while others \
# (like in 2018) are excluded, even those not listed as being possible stars. We will need to come back to this \
# in a later release...
#
# Discovered a major issue wit the Inada2010t3 table. I had not properly accounted for redshifts that were 0 or \
# negative flags, and I cut on dV. As far as I am aware, this is the only time I did this in that notebook. This \
# threw out a bunch of objects, and several of these DO match with the BQ lists. I have now output that list of \
# 'lost' objects as a separate .csv and I will load it in and match it against the MAC in the matching catalog \
# notebook. Evidently I did the same thing to the 2012 notebook. I've output another .csv file for those 'lost' \
# sources as well. I've double-checked the formatting of the Inada+ cells (and the input files) to make sure the \
# redshift columns are fine and that coordinate columns to not contain strings. I've also added in aseveral for \
# loops to overwrite bad redshift values and bad redshift type entries. 
#
# In the catalog matching notebook, I now have addition cells at the end that match the missing objects from Inada \
# 2010 and 2012 back in, and concatenates the remaining objects with the MAC main tables.
#
#
# found and fixed a bug in the match catalogs. I had indiv instead of indiv_x in a few instances when matching \
# the individual tables against the main table.

# In the X-shaped radio source noteook: corrected the seleciton method for one of the cheung2007 tables. I've \
# also just updated the cell that adds Roberts+ info to the Cheung+ targets. It now grabs the redshifts from \
# Roberts+ if there is no redshift from Cheung+.

# In the individual table, I've now added a redshift for NGC 1068
# Adjusted the matching stuff for the Barrows+2011 target so that a secondary z and name are not adopted
# Added 'bulge' and 'red blob' in name1 and name2 for the Markakis+ target

# Just realized that I was starting to adjust the RA and Dec entries in my notebooks (like marking duplicate RA \
# and Dec 2 as -99), but I CANNOT do that without messing up the total matching notebook, because then these \
# objects will be placed in the MACnoRA2 table and will be matched differently. 

#
# Just discovered an issue with the varstrometry table from Hwang+; I didn't realize it at the time, but they have \
# duplicate entries, because where objects had two GAIA matches, they placed the second match on a second row \
# instead of using separate columns... 
#    At this point, it might be easier to just go in and manually correct the secondary coordinates and drop the \
# 'duplicate rows'... we'll have to see...
#
#
# Another issue noticed: Ra and Dec values for gattano are -99 for some reason, but I can see matches with \
#    Inada+2010.... need to look into this issue...
#
# Corrected the coordinates for the Wang+2010 IRAS target. Also correcting the Pindor coordinates in the \
# individual table (since in the matching script I have it overwrite from the individual table)

# WE NEED TO DOUBLE CHECK WHY DV AND SEP IS NOT TAKEN FROM FU2012T3 AND T4!S
#

# Down below, I've now finished overwriting all of the duplicate Name2 values in the binary/recoil table (there \
# are some dual candidates int hat table that weren't piocked up by the other table, but that's okay)

# As of 18 December, I am now adding in a confidence flag column. It will start as -99, and change to our -1-->1 \
# system as we modify the flags for samples and individual targets


# pindor+2006 individual target, not overlapping with hennawi, has secondary coordinates relative to first



# Changes following 2 Jan 2024

# inada 2008 is missing the z types
# In Inada2008 table 2, I've fixed the missing z2 listing for SDSSJ100229.46+444942.7 (it is now -99)
# Within this notebook, I am going thorugh and by Name1 assigning the missing z type flags for Inada 2008 and other \
# papers where applicable

# just corrected the separation int he individual table for the Wong+2008 target. Sep is 1.3' or 78''. It was \
# listed as 1.3''. Still a dual based on our cosmology (~83 kpc)

# Corrections made to the lens botebook. Anguita2018 mistaken had inada table names listed in the \
# dV loop. I;ve commented out the selection and confirmation method column creation commands since I've \
# put these into the csv files themselves. For the Eftek2017 , Rusu2019, and Lemon2020 matches, I now have the code \
# take the redshifts from Lemon 2020.

# I have now gone through and added selection methods for all of the lens search papers to the actual csv files.\
# These may need to be refined, because pretty much all GAIA multi-peak selection methods are simply marked as \
# 'optical imaging' and nothing more specific, but we really should be more specific. 

# Confirmation methods have been added to the csv files for a few of these papers but not for all of them. Any \
# work that required looking through the original tables and differentiating between optical and IR imaging \
# typically does not have confirmation strategies listed, for now. (But they will be... next big thing...).
#

# I've now gone back into the matching notebooks notebook and adjusted the code where Inada gets matched to Gattano
# For some reaosn, the match I'm seeing looks correct. Which makes me think what I saw below and in some other cases
# was an issue of not resting the index. Looking back through my matching notebook, the index was **not** reset \
# after every concatenation with a new table, meaning we had a hell of a lot of duplicate indexes. I have now gone \
# through and added index reset commands throughout whenever I found a concatenation, and even when loading in \
# new tables that may not have been reset previously.

# I stand corrected. I'm not sure what I was seeing before... the match between Gattano and Inada looks fine after \
# all. In any event, I now pull most information over from Inada and really only add the Gattano citation and designation
#

# corrected the issue of a missing selection method and confirmation method for Rusu (for some reason had redshift listed instead)


# figure out why there is overlap on J002729.24+211152.08 between yuan and severgnini liu and serafinelli
# --> index was not being reset in the double peaked table just before adding this so it must have been added to \
# the original index as well as the yuan index. I now have added in an index resetting function.

# Indeed, once the index resetting was in place, the severgnini issue disappeared, as did the issue with the Kim2020
# non-matches and the issues with Smith, Song, and Kim

# I have now gone through and standardized (and fixed) the selection techniques in the binary notebook.
# Erac2013 was fixed *used to be listed as double-peak selected). I do not differentiate between broad Hbeta and \
# broad Mg II selection. This is something we could consider changing soon. I am also adding in 'Optical Spectroscopy' \
# and 'Fiber Spectroscopy' (or maybe Fiber optical spectroscopy).

# I am now debating whether LOS radial velocity and long-slit spec should be listed as a selection technique or \
# an analysis technique

# I have gone through and double-checked/correct all of the system types in the offset and Bl binary notebooks, \
# made sure a few columns were strings when they were supposed to be (such as brightness band and source), removed \
# source Name 2 and redshift 2 for a  umber of sources that did not require it, and I have revised the selection \
# techniques I believe not for all of the catalogs in those two notebooks. 

# In addition, I have now included all of the Mg II-only selected objects from Ju+2013 appropriately matched to \
# the remainder of the Wang+2017 catalog, and this table is now appropriately matched into the MAC at the end of \
# the matching notebook. 

# In the offset notebook, I now have also included the missing offset AGNs from comerford+2009 and the remainder \
# of Barrows+2016's sample (excluding anything I already tagged in the individual table or other notebooks such as \
# the Liu notebook). These two tables are saved and read directly into the matching notebook and appropriately \
# matched with the full MAC table. 

# As of now, I believe we have included all missing objects/major works. There are bound to be a few works that \
# we have missed, but I believe we have all of the major works and are probably only missing a few works on \
# binaries

# I have added 'radio imaging' to the Fu+2015 selection techniques and 'optical spectroscopy' to the Fu+2018 \
# objects. 

# Corrected a couplw of typos where we tack on the system types from eracleous+2012 and ju+2013. 

# Just removed the listing of 'Dual AGN Candidate / Offset AGN Candidate' tag within the double-peaked object ]
# notebook (this tag came from Comerford+2015)

# Corrected some typos in the Liu and Fu notebook... Fu's notes were being aded to the confirmation method, and the \
# Fu selection methods were overwriting the Liu selection methods

# Just a note: there are a few objects getting put into the bad , bad2, and bad3 frames but likely shouldn't be\
# something seems to be goingwrong when their separation in kpc or dV is calculated. need to double check. 


# as of today (17 Jan 2024), every object within the 'good' tables (as opposed to the ones listed as 'bad')
# have selection methodologies. This involved going through and formatting/reformatting a variety of individual \
# targets as well as removing '-99' listings in the selection method in the matching notebook and correcting \
# ahost of typos throughout the other notebooks (a few per notebook in the binary quasr, lens, etc. notebooks).
# Most of the work was revising/reformatting/double-checking the individual targets


# fixed a redshift issue that was introduced when we matched together Ianda+2008 and Mason+2000. \
#The Mason redshifts camme from simbad, drawn from SDSS but the redshifts likely suffered from fiber spillover \
# (they were very similiar) whereas the redshifts from Ianada 2008 were distinct (3000 km s^-1 difference.). \
# Therefore I have gone back and deferred to Inada for redshifts and coordinates

# We will be removing at least one of the gattano targets form the good set of tables

# moved wong+08 target to +1 in confidence. may revisit
# moved imanishi+2020 target back to a dual AGn candidate. not sure why that was a dual agn but the otherwasn't
# moved Woo+2014 down to dual agn candidate in individual table

# J048 fixed typo in matching catalog that put the wrong literature name for 1048 (j instead of i was used for \
# the MACnora2 index)



In [408]:
MAC = pd.read_csv('MAC_DR0p5_16Jan2024.csv', sep=',')

MAC.fillna(-99, inplace=True)

MAC['Confidence Flag'] = -99


In [409]:
#def find_matches(names):
#    name_dict = {}
#    matches = []
#
#    for name in names:
#        # Remove 'SDSS' or 'SDSS ' from the name
#        modified_name = name.replace('SDSS ', '').replace('SDSS', '')
#
#        first_6_chars = modified_name[:7]
#        if first_6_chars in name_dict:
#            matches.append((name_dict[first_6_chars], name))
#        else:
#            name_dict[first_6_chars] = name
#
#    return matches
#
#names = MAC['Name1'].to_list()
#matched_pairs = find_matches(names)
#print(len(matched_pairs))
#
#for pair in matched_pairs:
#    print(f"Match found: {pair[0]} and {pair[1]}")
    
    

In [410]:
# deprecating this list on 24 January 2024; I don't think we need to manually make this adjustment anymore
#bqoverlap = ['J115822.77+123518.5','J120727.09+140817.1','J123555.27+683627.0','J132022.54+305622.8',\
#             'J141855.41+244108.9','J142604.32+071930.0','J143002.88+071411.3','J145826.72+544813.1',\
#             'J150747.23+290333.2','J160603.02+290050.8','J163510.30+291116.1','J155330.23+223010.22']

for index, row in MAC.iterrows():
    # deprecating this hennawi line as of 24 january 2024,; I don't think it is necessary any longer
    #if 'Hennawi' in row['Paper(s)']:
    #    MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
    if 'J005113.94+002047.2' in row['Name1']:
        MAC.at[index, 'Processed System Type'] = 'Dual AGN' # this is from Fu+2015's sample
    elif 'J100602.14+071131.0' in row['Name1']:
        MAC.at[index, 'Processed System Type'] = 'Dual AGN' # from Rubinur's 2019 sample
    elif 'J220634.97+000327.6' in row['Name1']:
        MAC.at[index, 'Processed System Type'] = 'Dual AGN' # Fu+2015's sample
    elif 'SDSS J1120+6711A' in row['Name1']:
        MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
    # deprecated this next line on 24 january 2024; I don't think it is necessary anymore
    #elif row['Name1'] in bqoverlap:
    #    MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
    elif 'SDSS J171544.05+600835.7' in row['Name1']:
        MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate' 
        # we're reclassifying Julie's 2011 target from Dual AGN --> Dual AGN Candidate
    # this next line and the next line after are deprecated as of 24 Jan 2024 (no longer needed)
    #elif row['Processed System Type']=='Dual AGN Candidate / Offset AGN Candidate':
    #    MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
    #    # here we're overwriting Julie's offset AGN classification, because in all cases they were also dual candidates
    #    # we may need to come back and think about this some more, because Julie's work was selecting dual SMBH candidates \
    #    # within a single host galaxy, but some of the overlap (ex. with Xin Liu's targets) have large separations \
    #    # (ex. 70 kpc), so these targets are technically a dual within a dual.... but I don't think I trust that offset \
    #    # AGNs truly are tracing dual SMBHs. We can always come back and change this later.
    #elif row['Processed System Type']=='Dual AGN Candidate / Likely Single AGN':
    #    MAC.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
    #    #'Dual AGN Candidate / Likely Single AGN' is both a dual AGN candidate based on Liu+2011 (so resolved spec pair),\
    #    # but ALSO a dual AGN candidate based on double-peak selection (Wang, and the followed up by others). \
    #    # Muller-sanchez followed up with radio and showed that the origin of the double-peak was still ambiguous, but \
    #    # all of the papers seem to point to the double-peaked AGN being a single AGN. Muller-sanchez's imaging did not \
    #    # cover both nuclei that Liu+ consider to be a pair, so the folks doing the double-peaked work did not focus on ]
    #    # the companion at ~6'' away. So we'll class this as a dual AGN candidate
    #    #
    #    # Users will have to be wary of these kinds of cases, because the subjective flag will probably be +0.5 here,\
    #    # but that will reflect the dual candidacy from Liu's work and NOT the candidacy of the double-peaked work
    #    #
    #J155330.23+223010.22 diff redshifts
    #J163510.30+291116.1 diff redshifts?, we'll handle this later
    #J150747.23+290333.2 diff redshifts but doesnt meet dV cut; we'll remove this later, put it int he bad table, and relabel as clustered QSOs
    #J145826.72+544813.1 diff z
    # J143002.88+071411.3 diff z
    # J142604.32+071930.0 diff z
    # J141855.41+244108.9 diff z?, we'll handle this later
    # J123555.27+683627.0 diff z 
    
#pippin = MAC[MAC['Processed System Type']=='Dual AGN / Dual AGN Candidate / Recoil Candidate']

# Binary AGN Candidate / Dual AGN --> This is J09527, McGurk+2011's target that was also selected by Liu+2014
# Dual AGN Candidate / Recoil Candidate --> overlap between x-shaped radio sources and double peak and a couple individual targets
# Binary AGN Candidate / Dual AGN Candidate this is fine
# Binary AGN Candidate / Dual AGN Candidate / Recoil Candidate this is fine and is due to X-shaped radio source overlap
# Binary SMBH Candidate / Dual AGN Candidate --> just one target, and is fine


### Fixed:
# 'Dual AGN / Offset AGN Candidate' --> Originally from Liu, followed-up by husemann. Also selected by Barrows as \
# offset AGN, but I think after some reading, it will prob be marked as a dual AGN only.
# 'Dual AGN / Dual AGN Candidate' --> a lot of overlap between Hennawi and others, and then some with Liu2011 \
# and others
#'Dual AGN / Likely Single AGN' --> This is Julie Comerford's 2011 target, followed up by Liu, Smith, Smith, and others
# 'Dual AGN Candidate / Offset AGN Candidate' --> overlap between comerford+ and liu+2011, inada+2010, smith2010, \
# and liu2010
# 'Dual AGN Candidate / Likely Single AGN' see notes above
# 'Binary AGN Candidate / Binary SMBH Candidate / Dual AGN Candidate' --> This is J1536+0441
# Looking at how amyn people grabbed this target and for different reasons, I'm okay keeping this cross class

# note: when we add the Fu+ selection method, we overwrite the liu2011 selection method. make sure to fix this in the notebook.
# J005113.94+002047.2 --> confirmed binary in radio.
# J220634.97+000327.6 --> confirmed binary in radio.
# J100602.14+071131.0 --> I believe this is confirmed as a dual in the radio by rubinur but confirm
# SDSS J1120+6711A --> likely dual based on Pindor's work
# anything with hennawi+ gets pushed to dual agn status. 

# 2345+007 --> confirmed dual AGN ...
# myers+ targets, unless we have redshifts for both, are staying as dual candidates. If we have redshifts for \
# both and there is some dV, we'll consider them as duals with high confidence


#pippin

In [411]:
# here I'm going to make a legacy column called 'legacy system type'
# I'll use this later oon when I want to come back and add in the legacy system types that others have used, like \
# 'binary quasar'

MAC['Legacy System Type'] = MAC['System Type'].astype(str)

# and now here we're going to replace all 'binary quasar' system types with 'dual AGNs', and simialrly for the \
# candidates...

types = MAC['System Type'].dropna().str.split(' / ')

# Step 2 and 3: Remove duplicates, alphabetize, and replace 'binary quasar' with 'dual AGN' in any context for each cell
def process_cell(cell):
    # Replace 'binary quasar' with 'dual AGN' in any context
    cell = [x.replace('Binary Quasar Candidate', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Binary Quasar', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Quasar Pair', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Dual AGN Candidate / Likely Single AGN', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Likely Single AGN', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Single AGNs', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Single AGN', 'Dual AGN Candidate') for x in cell]
    cell = [x.replace('Dual AGN Candidate / Likely Single AGN', 'Dual AGN Candidate') for x in cell]
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

# Step 4: Join the contents back into a single string
MAC['Processed System Type'] = processed_types.apply(' / '.join)


In [412]:
#gandalf = MAC[MAC['Processed System Type'].str.contains('Dual AGN', na=False)]

unique_combinations = MAC['Processed System Type'].dropna().unique()

print(unique_combinations)
print(len(unique_combinations))


['Dual AGN Candidate' 'Dual AGN / Dual AGN Candidate'
 'Dual AGN Candidate / Dual SMBH Candidate'
 'Dual AGN Candidate / Recoil Candidate' 'Dual AGN'
 'Binary AGN Candidate / Dual AGN Candidate'
 'Binary SMBH Candidate / Dual AGN Candidate'
 'Binary AGN Candidate / Binary SMBH Candidate / Dual AGN Candidate'
 'Binary AGN Candidate' 'Binary AGN Candidate / Binary SMBH Candidate'
 'Dual SMBH Candidate'
 'Binary AGN Candidate / Dual SMBH Candidate / Recoil Candidate'
 'Binary AGN Candidate / Dual SMBH Candidate' 'Binary SMBH Candidate'
 'Recoil Candidate' 'Binary AGN Candidate / Recoil Candidate'
 'Binary AGN Candidate / Dual AGN Candidate / Recoil Candidate'
 'Binary AGN Candidate / Binary SMBH Candidate / Dual AGN Candidate / Recoil Candidate'
 'Binary AGN'
 'Binary SMBH Candidate / Dual SMBH Candidate / Recoil Candidate'
 'Binary SMBH Candidate / Dual AGN'
 'Binary SMBH Candidate / Recoil Candidate'
 'Binary SMBH Candidate / Dual AGN Candidate / Recoil Candidate']
23


In [413]:
# here we are going to implement a system type column for primary, secondary, and tertiary system types \
#for better organization

# Object type lists
dualcantype = ['Dual AGN','Dual AGN Candidate','Dual SMBH Candidate']
binarycandtype = ['Binary AGN','Binary SMBH','Binary AGN Candidate','Binary SMBH Candidate']
recoiltype = ['Recoil Candidate']

# Function to check items in the lists and assign labels
def assign_types(row):
    types = row['Processed System Type'].split(' / ')
    primary, secondary, tertiary = '', '', ''
    # Check against dualcantype
    primary_matches = [item for item in types if item in dualcantype]
    if primary_matches:
        primary = ' ; '.join(primary_matches)
        # now check against bianry types
        secondary_matches = [item for item in types if item in binarycandtype]
        if secondary_matches:
            secondary = ' ; '.join(secondary_matches)
            # now check against recoil types
            tertiary_matches = [item for item in types if item in recoiltype]
            if tertiary_matches:
                tertiary = ' ; '.join(tertiary_matches)
            else:
                tertiary = "-99"
        # and checking againast recoil types in the event thatno binary types were detected
        else:
            secondary_matches = [item for item in types if item in recoiltype]
            if secondary_matches:
                secondary = ' ; '.join(secondary_matches)
                tertiary = "-99"
            else:
                secondary = "-99"
                tertiary = "-99"
    else:
        # Check against binarycandtype if dualcantype had no matches
        secondary_matches = [item for item in types if item in binarycandtype]
        if secondary_matches:
            primary = ' ; '.join(secondary_matches)
            secondary_matches = [item for item in types if item in recoiltype]
            if secondary_matches:
                secondary = ' ; '.join(secondary_matches)
                tertiary = "-99"
            else:
                secondary = "-99"
                tertiary = "-99"
        else:
            # Check against recoiltype if previous lists had no matches
            tertiary_matches = [item for item in types if item in recoiltype]
            if tertiary_matches:
                primary = ' ; '.join(tertiary_matches)
                secondary = "-99"
                tertiary = "-99"
                
    return pd.Series([primary, secondary, tertiary])

# Apply the function to each row
MAC[['Primary System Type', 'Secondary System Type', 'Tertiary System Type']] = MAC.apply(assign_types, axis=1)

for index, row in MAC.iterrows():
    if 'Dual AGN' in row['Primary System Type'].split(' ; '):
        #print('True')
        MAC.at[index, 'Primary System Type'] = 'Dual AGN'

# and adding in the confidence rank columns for each system type column
MAC['ST1 Confidence Flag'] = -99
MAC['ST2 Confidence Flag'] = -99
MAC['ST3 Confidence Flag'] = -99


In [414]:
#MAC

In [415]:
MAC_names = MAC['Name1'].to_list()

# adding separations to binary candidates

runnoe = ['001224.02-102226.2','015530.01-085704.0','031715.87+425150.6','074007.28+410903.6','082150.13+081907.3',\
          '091928.69+143202.6','092712.65+294344.0','093844.45+005715.7','094603.94+013923.7','094620.32+375953.2',\
          '095036.75+512838.1','095539.81+453217.0','105041.36+345631.4','110556.18+031243.2',\
          '111537.93+542725.2','113330.29+105223.2','113904.33+465651.1','115158.90+122128.9','120924.07+103612.1',\
          '125142.28+240435.3','131945.95+053002.7','140251.19+263117.5','151132.53+100953.1','153636.22+044127.0',\
          '153644.90+141229.7','154637.12+122832.5','155654.47+253233.5','161911.24+501109.2','162914.09+151415.3']
runnoesep = [2.6e-5,1.1e-5,1.7e-5,1.0e-5,1.1e-5,1.3e-5,1.3e-5,1.6e-5,1.5e-5,1.5e-5,0.6e-5,0.9e-5,8.1e-5,1.3e-5,\
              3.3e-5,1.7e-5,3.2e-5,0.1e-5,1.4e-5,2.9e-5,0.6e-5,2.1e-5,1.7e-5,13.4e-5,1.2e-5,1.1e-5,5.7e-5,0.2e-5,\
              0.7e-5]
# those are minimum separations

#for i in runnoe:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')

for i, j in zip(runnoe,runnoesep):
    MAC.loc[MAC.Name1==str(i), 'Sep(kpc)'] = j*(1.e-3)
    #for index, row in MAC.iterrows():
    #    if row['Name1']==str(i):
    #        MAC.at[index, 'Sep(kpc)'] = j*(1.e-3)

MAC.loc[MAC.Name1=='001224.01-102226.5', 'Sep(kpc)'] = (2.6e-2)*(1.e-3)
MAC.loc[MAC.Name1=='094603.94+013923.6', 'Sep(kpc)'] = (1.5e-2)*(1.e-3)
MAC.loc[MAC.Name1=='J1050+3456', 'Sep(kpc)'] = (8.1e-2)*(1.e-3)
MAC.loc[MAC.Name1=='J154637.12+122832.5', 'Sep(kpc)'] = (1.1e-2)*(1.e-3)
            
# 001224.02 --> overlap with liu but liu does not have seps. can overwrite
# 094603 --> overlap with liu but can overwrite
# 105041 --> overlap with tsalmantza can overwrite (1050+3456)

# 092712 --> this is 0927 do not overwrite?
# 153636 --> this is 1536 do not overwrite

#(these do match via name)
# 154637--> overlap with smith can overwrite
# 095036.75+512838.1 --> overlap wth liu but can overwrite
# 154637.12+122832.5 --> overlap with smith and can overwrite


#ju2013 = ['J032223.02-000803.5','J002444.11+003221.4','J095656.42+535023.2','J161609.50+434146.8',\
#          'J075700.70+424814.5','J093502.54+433110.7','J004918.98+002609.4']
#
#ju2013seps = [0.032,0.102,1.74,0.021,0.020,0.181,0.096]
# those are maximum separations

print(' ')

ju2013tab = pd.read_csv('Tables/Ju2013/Ju2013_t2.csv', sep=',')
ju2013names = ju2013tab['SDSS'].to_list()
ju2013sep = ju2013tab['rmax(pc)'].to_list()

#for i in ju2013names:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')
 
for i, j in zip(ju2013names,ju2013sep):
    MAC.loc[MAC.Name1==str(i), 'Sep(kpc)'] = j*(1.e-3)
    #for index, row in MAC.iterrows():
    #    if row['Name1']==str(i):
    #        MAC.at[index, 'Sep(kpc)'] = j*(1.e-3)

# and fixing the one lonesome object that overlapped with Eracleous' sample
MAC.loc[MAC.Name1=='J002444.11+003221.4', 'Sep(kpc)'] = 0.000102
        
#for i in ju2013:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')
#        # the one that isn't in the table here is the same as the overlaping target from our first loop
        
# first one overlaps with eraclerous but we can adopt the ju sep
# J020646.97+001800.6 is the overlap with hennawi, so we will not overwrite the separation


# now working on the 9 binary candidates from Liu+2014
liu2014 = ['082930.59+272822.7','084716.03+373218.0','085237.01+200410.9','092837.98+602521.0',\
           '103059.09+310255.7','110050.99+170934.2','111230.89+181311.4','130534.49+181932.8',\
           '134548.50+114443.5']
liu2014sep = [0.24,0.29,0.13,0.46,0.25,0.092,0.12,0.10,0.11]
# these are maximum values drawn from the q=2 column of Liu+2014 table 3

#for i in liu2014:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')
#        # here we're going to be adding in the separation value from Ju+2013
#        # I've gone through and verified that we will not be overwriting any separations when we do this

for i, j in zip(liu2014,liu2014sep):
    MAC.loc[MAC.Name1==str(i), 'Sep(kpc)'] = j*(1.e-3)
    #for index, row in MAC.iterrows():
    #    if row['Name1']==str(i):
    #        MAC.at[index, 'Sep(kpc)'] = j*(1.e-3)

# now for Graham2015
graham2015 = ((Table.read('Tables/Graham2015/table2.dat', readme = 'Tables/Graham2015/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
graham2015names = graham2015['Name'].to_list()
graham2015sep = graham2015['r'].to_list()

#for i in graham2015names:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')

for i, j in zip(graham2015names,graham2015sep):
    MAC.loc[MAC.Name1==str(i), 'Sep(kpc)'] = j*(1.e-3)
    #for index, row in MAC.iterrows():
    #    if row['Name1']==str(i):
    #        MAC.at[index, 'Sep(kpc)'] = j*(1.e-3)

MAC.loc[MAC.Name1=='PG 1302-2102 / BZQJ1305-1033', 'Sep(kpc)'] = 0.008*(1.e-3)
# ther are four objects that overlap with other samples
# --> one overlaps with Liu2014 (RXS J10304+5516,0.435,spec,10:30:25.0,+55:16:23.4). Liu doesn;t calc sep, so we can overwtite
# SDSS J081617.73+293639.6 not in list. --> SDSSJ081617.73+293639.6 AGN1  (overlap with Ianda, we will not defer)
# BZQJ1305-1033 not in list. -->This is PG 1302-2102, we will defer to Graham, for now. For some reason this was 0.01 before
# SDSS J153636.22+044127.0 not in list. --> J1536+0441 / SDSS J153636.22+044127.0 (do not overwrite sep)

# now for Charisi2016
charisi2016 = pd.read_csv('Tables/Charisi2016/Charisi2015_t2.csv', sep=',')
charisi2016names = charisi2016['Name'].to_list()
charisi2016sep = charisi2016['Angular'].to_list()

#for i in charisi2016names:
#    if i not in MAC_names:
#        print(str(i)+' not in list.')

for i, j in zip(charisi2016names,charisi2016sep):
    MAC.loc[MAC.Name1==str(i), 'Sep'] = j*(1.e-6)
    #for index, row in MAC.iterrows():
    #    if row['Name1']==str(i):
    #        MAC.at[index, 'Sep'] = j*(1.e-6)

# we're missing the last object in the charisi2016. should be no matches                       


# and now adding in the info from Guo+2019 (and superceding info from Liu+2014 when applicable

#J0322+0055 not in liu2014 binary list but in liu2014 and shen 2013
#J0829+2728 in liu2014
#J0847+3732 in liu2014
#J0852+2004 in liu2014
#J0928+6025 in liu2014
#J1030+3102 in liu2014
#J1100+1709 in liu2014
#J1112+1813 in liu2014
#J1410+3643 not in liu2014 binary list but in shen 2013
#J1537+0055 not in liu2014 binary list but in shen 2013
#J1550+0521 not in liu2014 binary list but in shen 2013
#J2349-0036 not in liu2014 binary list but in shen 2013

guo2019 = ['032213.89+005513.4','082930.60+272822.7','084716.04+373218.1','085237.02+200411.0','092837.98+602521.0',\
           '103059.09+310255.8','110051.02+170934.3','111230.90+181311.4','141020.57+364322.7',\
           '153705.95+005522.8','155053.16+052112.1','234932.77-003645.8']

guo2019sep = [0.056,0.070,0.088,0.11,0.10,0.072,0.059,0.050,0.050,0.039,0.061,0.072]
# these are the larger of the two values listed in Guo+2019's table (derived from q=0.5)

for i, j in zip(guo2019,guo2019sep):
    MAC.loc[MAC.Name1==str(i), 'Sep(kpc)'] = j*(1.e-3)

# and now fixing anything marked as having zero sep from Graham+2015:
MAC.loc[MAC['Sep(kpc)']==0, 'Sep(kpc)'] = -99

# once we're done here, we'll need to come back later and finally add in the 'upper/lower limit' flag


# check with Nathan what he thinks for Hwang+2020 and Orosz+2013


In [416]:
# here we're going to calculate angular separations for systems where we only have physical reported separations...

missingsep = MAC[(MAC['Sep(kpc)']>0) & (MAC['Sep']<0)]
objs = missingsep['Name1'].to_list()
for i in objs:
    for index, row in MAC[MAC['Name1']==str(i)].iterrows():
        MAC.at[index, 'Sep'] = (row['Sep(kpc)']/(cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)))

## here we're fixing the known issues in the gimli table for PG 1302, 3C 293, WISE J2332, PKS 0301, SDSS J059, 
## and PSO J334
#
#objs = ['PG 1302-2102 / BZQJ1305-1033','3C 293','WISE J233237.05-505643.5 (W2332-5056)','PKS 0301-243',\
#        'SDSS J0159+0105','PSO J334.2028+01.4075']
#for i in objs:
#    for index, row in MAC[MAC['Name1']==str(i)].iterrows():
#        MAC.at[index, 'Sep'] = (row['Sep(kpc)']/(cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)))
#    
## and now for the gandalf targets:
#objs = ['MGB2016+112 Core 1']
#
#for i in objs:
#    for index, row in MAC[MAC['Name1']==str(i)].iterrows():
#        MAC.at[index, 'Sep'] = (row['Sep(kpc)']/(cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)))

     
# and now for IRAS 20210; this one has distinct coordinates, so we will reomcpute the angular separation \
# based on the XMM souce positions
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

# note here I went back and used panstarrs in aladin lite to get the optical positions of the nuclei and overwrote \
# the listed positions in the individual table because those were for the extraction regions used by Piconcelli \
# and do not correspond to the true X-ray peaks. The panstarrs coordinates give a slightly higher separation \
# (~12.3 kpc) than reported by pinconcelli (~11kpc)
objs = ['I20210N']
for i in objs:
    for index, row in MAC[MAC['Name1']==str(i)].iterrows():
        c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
        c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
        MAC.at[index, 'Sep'] = c1.separation(c2).arcsecond


#for index, row in gandalf_white.iterrows():
#    gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#    gandalf_white.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#    gandalf_white.at[index, 'Sep(kpc)_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)

#MAC.loc[MAC.Name1=='I20210N']


In [417]:
#nan_rows_in_A = MAC[MAC['Sep'].isna()]
#nan_rows_in_A

In [418]:
#MACcheck = MAC[(MAC['Sep(kpc)']<1) & (MAC['Sep(kpc)']>-10)]
#len(MACcheck)
#MACcheck

In [419]:
# now we're going to go through and make plenty of fine adjustments, including calculating separations and \
# adding missing information 

# adding in additional coordinate information here
MAC.loc[MAC.Name1=='ESO 1327-2041 Galaxy', 'RA2'] = '13:30:06.087'
MAC.loc[MAC.Name1=='ESO 1327-2041 Galaxy', 'Dec2'] = '-20:55:19.84'
MAC.loc[MAC.Name1=='ESO 1327-2041 Galaxy', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='ESO 1327-2041 Galaxy', 'Coordinate_waveband2'] = 'Optical'
MAC.loc[MAC.Name1=='ESO 1327-2041 Galaxy', 'Coordinate_Source2'] = 'PanSTARRS'
#ESO 1327 Ejected Nucleus 13:30:06.087 -20:55:19.84
#optical panstarrs j2000

#VV 114
MAC.loc[MAC.Name1=='VV 114 E', 'RA1'] = '01:07:47.595'
MAC.loc[MAC.Name1=='VV 114 E', 'Dec1'] = '-17:30:24.09'
MAC.loc[MAC.Name1=='VV 114 E', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='VV 114 E', 'Coordinate_waveband1'] = 'Optical'
MAC.loc[MAC.Name1=='VV 114 E', 'Coordinate_Source1'] = 'PanSTARRS'
MAC.loc[MAC.Name2=='VV 114 W', 'RA2'] = '01:07:46.551'
MAC.loc[MAC.Name2=='VV 114 W', 'Dec2'] = '-17:30:22.27'
MAC.loc[MAC.Name2=='VV 114 W', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name2=='VV 114 W', 'Coordinate_waveband2'] = 'Optical'
MAC.loc[MAC.Name2=='VV 114 W', 'Coordinate_Source2'] = 'PanSTARRS'
    
#PKS B1345+125 NW / PKS B1345+125 SE
MAC.loc[MAC.Name1=='PKS B1345+125 NW', 'RA2'] = '13:47:33.481'
MAC.loc[MAC.Name1=='PKS B1345+125 NW', 'Dec2'] = '+12:17:23.86'
MAC.loc[MAC.Name1=='PKS B1345+125 NW', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='PKS B1345+125 NW', 'Coordinate_waveband2'] = 'Optical'
MAC.loc[MAC.Name1=='PKS B1345+125 NW', 'Coordinate_Source2'] = 'PanSTARRS'     

# SBS 1421+511
MAC.loc[MAC.Name1=='SBS 1421+511 QSO', 'RA2'] = '14:23:14.202'
MAC.loc[MAC.Name1=='SBS 1421+511 QSO', 'Dec2'] = '+50:55:40.39'
MAC.loc[MAC.Name1=='SBS 1421+511 QSO', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='SBS 1421+511 QSO', 'Coordinate_waveband2'] = 'Optical'
MAC.loc[MAC.Name1=='SBS 1421+511 QSO', 'Coordinate_Source2'] = 'PanSTARRS'  

# J1536+0441
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'RA1'] = '15:36:36.2232'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Dec1'] = '+04:41:27.069'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Coordinate_waveband1'] = 'Radio'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Coordinate_Source1'] = 'EVN'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'RA2'] = '15:36:36.2881'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Dec2'] = '+04:41:27.054'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Coordinate_waveband2'] = 'Radio'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Coordinate_Source2'] = 'EVN'
MAC.loc[MAC.Name1=='J1536+0441 / SDSS J153636.22+044127.0', 'Name1'] = 'J1536+0441 VLA-A'
MAC.loc[MAC.Name1=='J1536+0441 VLA-A', 'Name2'] = 'J1536+0441 VLA-B'
# these radio positions come from Bondi+2010 and adopt the naming convention of Wrobel+2009

#Adjusted NGC 7592 E coordinates manually. Original coordinates sat between galaxies

# SDSSJ1600+0000A
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'RA1'] = '16:00:15.506'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Dec1'] = '+00:00:45.43'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Coordinate_waveband1'] = 'Optical'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Coordinate_Source1'] = 'PanSTARRS'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'RA2'] = '16:00:15.591'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Dec2'] = '+00:00:46.70'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Coordinate_waveband2'] = 'Optical'
MAC.loc[MAC.Name1=='SDSSJ1600+0000A', 'Coordinate_Source2'] = 'PanSTARRS'
#c1 = SkyCoord(Angle('16:00:15.506 hours').degree,Angle('+00:00:45.43 degrees').degree, unit='deg', frame='icrs')
#c2 = SkyCoord(Angle('16:00:15.591 hours').degree,Angle('+00:00:46.70 degrees').degree, unit='deg', frame='icrs')

# need to get coordinates for 3C 186 Quasar (in gimli table)




# now looping over double-peaked objects that we have ocordinates for
# now the single AGNs from Comerford+2015
MAC.loc[MAC.Name1=='J014209-005049', 'RA1'] = '01:42:09.003'
MAC.loc[MAC.Name1=='J014209-005049', 'Dec1'] = '-00:50:49.94'
MAC.loc[MAC.Name1=='J014209-005049', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J014209-005049', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J014209-005049', 'Coordinate_Source1'] = 'HST WFC3'

MAC.loc[MAC.Name1=='J085416.76+502632.0', 'RA1'] = '08:54:16.776'
MAC.loc[MAC.Name1=='J085416.76+502632.0', 'Dec1'] = '+50:26:32.10'
MAC.loc[MAC.Name1=='J085416.76+502632.0', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J085416.76+502632.0', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J085416.76+502632.0', 'Coordinate_Source1'] = 'HST WFC3'

MAC.loc[MAC.Name1=='J100654.20+464717.2', 'RA1'] = '10:06:54.237'
MAC.loc[MAC.Name1=='J100654.20+464717.2', 'Dec1'] = '+46:47:16.71'
MAC.loc[MAC.Name1=='J100654.20+464717.2', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J100654.20+464717.2', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J100654.20+464717.2', 'Coordinate_Source1'] = 'HST WFC3'

MAC.loc[MAC.Name1=='J144804.17+182537.9', 'RA1'] = '14:48:04.186'
MAC.loc[MAC.Name1=='J144804.17+182537.9', 'Dec1'] = '+18:25:37.75'
MAC.loc[MAC.Name1=='J144804.17+182537.9', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J144804.17+182537.9', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J144804.17+182537.9', 'Coordinate_Source1'] = 'HST WFC3'

# now for the dual'offset' candidates from Comerford+2015
MAC.loc[MAC.Name1=='J0952+2552', 'RA1'] = '09:52:07.611'
MAC.loc[MAC.Name1=='J0952+2552', 'Dec1'] = '+25:52:57.23'
MAC.loc[MAC.Name1=='J0952+2552', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J0952+2552', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J0952+2552', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J0952+2552', 'RA2'] = '09:52:07.604'
MAC.loc[MAC.Name1=='J0952+2552', 'Dec2'] = '+25:52:56.24'
MAC.loc[MAC.Name1=='J0952+2552', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J0952+2552', 'Coordinate_waveband2'] = 'F160W'
MAC.loc[MAC.Name1=='J0952+2552', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J0952+2552', 'Literature Name'] = 'J09527.62+255257.2'
MAC.loc[MAC.Name1=='J0952+2552', 'Name1'] = 'J09527.62+255257.2 / J0952+2552NE'
MAC.loc[MAC.Name1=='J09527.62+255257.2 / J0952+2552NE', 'Name2'] = 'J0952+2552SW'

MAC.loc[MAC.Name1=='J123915.40+531414.6', 'RA1'] = '12:39:15.454'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Dec1'] = '+53:14:15.14'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'RA2'] = '12:39:15.454'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Dec2'] = '+53:14:15.14'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Coordinate_waveband2'] = 'F160W'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J123915.40+531414.6', 'Name1'] = 'J123915.40+531414.6 / J1239+5314NE'
MAC.loc[MAC.Name1=='J123915.40+531414.6 / J1239+5314NE', 'Name2'] = 'J1239+5314SW'

MAC.loc[MAC.Name1=='J132231.86+263159.1', 'RA1'] = '13:22:31.796'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Dec1'] = '+26:31:58.65'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'RA2'] = '13:22:31.968'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Dec2'] = '+26:31:59.02'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Coordinate_waveband2'] = 'F160W'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J132231.86+263159.1', 'Name1'] = 'J132231.86+263159.1 / J1322+2631SW'
MAC.loc[MAC.Name1=='J132231.86+263159.1 / J1322+2631SW', 'Name2'] = 'J1322+2631NE'

MAC.loc[MAC.Name1=='J135646.11+102609.1', 'RA1'] = '13:56:46.128'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Dec1'] = '+10:26:08.61'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'RA2'] = '13:56:46.120'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Dec2'] = '+10:26:07.29'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Coordinate_waveband2'] = 'F160W'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J135646.11+102609.1', 'Name1'] = 'J135646.11+102609.1 / J1356+1026NE'
MAC.loc[MAC.Name1=='J135646.11+102609.1 / J1356+1026NE', 'Name2'] = 'J1356+1026SW'

MAC.loc[MAC.Name1=='J112659.54+294442.8', 'RA1'] = '11:26:59.569'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Dec1'] = '+29:44:42.63'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Coordinate_waveband1'] = 'F160W'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'RA2'] = '11:26:59.631'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Dec2'] = '+29:44:41.92'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Coordinate_waveband2'] = 'F160W'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J112659.54+294442.8', 'Name1'] = 'J112659.54+294442.8 / J1126+2944NW'
MAC.loc[MAC.Name1=='J112659.54+294442.8 / J1126+2944NW', 'Name2'] = 'J1126+2944SE'


# now for the objects in Liu+2013 and Liu+2018
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'RA1'] = '11:08:51.029'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Dec1'] = '+06:59:01.32'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Coordinate_waveband1'] = 'Y-band'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'RA2'] = '11:08:51.061'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Dec2'] = '+06:59:00.81'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Coordinate_waveband2'] = 'Y-band'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J110851.04+065901.4', 'Name1'] = 'J110851.04+065901.4 / J1108+0659NW'
MAC.loc[MAC.Name1=='J110851.04+065901.4 / J1108+0659NW ', 'Name2'] = 'J1108+0659SE'

MAC.loc[MAC.Name1=='J113126.08-020459.2', 'RA1'] = '11:31:26.042'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Dec1'] = '-02:04:59.33'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Coordinate_waveband1'] = 'Y-band'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'RA2'] = '11:31:26.088'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Dec2'] = '-02:04:59.21'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Coordinate_waveband2'] = 'Y-band'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J113126.08-020459.2', 'Name1'] = 'J113126.08-020459.2 / J1131-0204W'
MAC.loc[MAC.Name1=='J113126.08-020459.2 / J1131-0204W', 'Name2'] = 'J1131-0204E'

MAC.loc[MAC.Name1=='J114642.47+511029.6', 'RA1'] = '11:46:42.466'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Dec1'] = '+51:10:29.46'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Coordinate_waveband1'] = 'Y-band'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'RA2'] = '11:46:42.630'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Dec2'] = '+51:10:31.69'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Coordinate_waveband2'] = 'Y-band'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J114642.47+511029.6', 'Name1'] = 'J114642.47+511029.6 / J1146+5110SW'
MAC.loc[MAC.Name1=='J114642.47+511029.6 / J1146+5110SW', 'Name2'] = 'J1146+5110NE'

MAC.loc[MAC.Name1=='J133226.34+060627.4', 'RA1'] = '13:32:26.340'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Dec1'] = '+06:06:27.31'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Coordinate_waveband1'] = 'Y-band'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'RA2'] = '13:32:26.372'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Dec2'] = '+06:06:28.73'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Coordinate_waveband2'] = 'Y-band'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J133226.34+060627.4', 'Name1'] = 'J133226.34+060627.4 / J1332+0606SW'
MAC.loc[MAC.Name1=='J133226.34+060627.4 / J1332+0606SW', 'Name2'] = 'J1332+0606NE'

MAC.loc[MAC.Name1=='J092455.24+051052.0', 'RA1'] = '09:24:55.277'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Dec1'] = '+05:10:52.09'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Equinox1'] = 'J2000'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Coordinate_waveband1'] = '[OIII]'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Coordinate_Source1'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'RA2'] = '09:24:55.255'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Dec2'] = '+05:10:52.13'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Equinox2'] = 'J2000'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Coordinate_waveband2'] = '[OIII]'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Coordinate_Source2'] = 'HST WFC3'
MAC.loc[MAC.Name1=='J092455.24+051052.0', 'Name1'] = 'J092455.24+051052.0 / J0924+0510E'
MAC.loc[MAC.Name1=='J092455.24+051052.0 / J0924+0510E', 'Name2'] = 'J0924+0510W'


# tagging some additional DOI info I missed previously for J135429.06+132757.3 (tagged by Liu and Keel)
MAC.loc[MAC.Name1=='J135429.06+132757.3', 'Paper(s)'] += ' ; Keel+2019'
MAC.loc[MAC.Name1=='J135429.06+132757.3', 'BibCode(s)'] += ' ; 2019MNRAS.483.4847K'
MAC.loc[MAC.Name1=='J135429.06+132757.3', 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty3332'

# adding missing angular separations now...
objs = ['J1558+2723 G3','IRAS 05589+2828','IRAS 03219+4031','J181611.72+423941.6 (J1816 NE)','Mrk 248',\
        'Mrk 268','NGC 7679','M106','NGC 3227','NGC 2992','NGC 1052','MCG +04-48-002','4C60.07 Radio Core',\
       'J1114+4036 A','ESO 286-G17','ESO 432-IG006 SW / WISEA J084427.19-314150.8','J1036+0221','J0122+0100 NW',\
       'J1045+3519 W','J1221+1137 NE','J1301+2918 NE','SDSS J084810.10+351534.4','NGC 5278','UGC 6081',\
       'NGC 7592 E','ESO 1327-2041 Galaxy','J1536+0441 VLA-A','SBS 1421+511 QSO','PKS B1345+125 NW','VV 114 E',\
       'J0122+0100 NW','J0841+0101 E','J0859+1310 NE','J0905+3747 NE','J1045+3519 W','J1159+5320 SE',\
       'J1221+1137 NE','J1301+2918 NE','J2356-1016 NW','SDSSJ1600+0000A','J09527.62+255257.2 / J0952+2552NE',\
       'J123915.\40+531414.6 / J1239+5314NE','J132231.86+263159.1 / J1322+2631SW',\
        'J135646.11+102609.1 / J1356+1026NE','J112659.54+294442.8 / J1126+2944NWJ112659.54+294442.8',\
        'J110851.04+065901.4 / J1108+0659NW','J113126.08-020459.2 / J1131-0204W',\
        'J114642.47+511029.6 / J1146+5110SW',\
        'J133226.34+060627.4 / J1332+0606SW','J092455.24+051052.0','J020954.80-100223.00']

for i in objs:
    try:
        for index, row in MAC[MAC['Name1']==str(i)].iterrows():
            c1 = SkyCoord(Angle(str(row['RA1'])+' hours').degree,Angle(str(row['Dec1'])+' degrees').degree, unit='deg', frame='icrs')
            c2 = SkyCoord(Angle(str(row['RA2'])+' hours').degree,Angle(str(row['Dec2'])+' degrees').degree, unit='deg', frame='icrs')
            MAC.at[index, 'Sep'] = c1.separation(c2).arcsecond
    except:
        print(str(i))


# and now we're overwriting any junk where I previously put 0 for a separation
MAC.loc[MAC.Sep==0, 'Sep'] = -99

# J1558+2723 G3 Eckert+2017
# 4C60.07 Radio Core --> sedgwick target
# BAT selected in Koss 2012:
# 'IRAS 05589+2828','IRAS 03219+4031','J181611.72+423941.6 (J1816 NE)','Mrk 248',\
#        'Mrk 268','NGC 7679','M106','NGC 3227','NGC 2992','NGC 1052','MCG +04-48-002'
#J1114+4036 A --> barrows 2016
# ESO 286-G17 --> Sekiguchi+1992
# ESO 432-IG006 SW / WISEA J084427.19-314150.8 --> Torres-Alba+2018
# J1036+0221 --> Satyapal, Dutta, and Pfeifle
# Batcheldor+2010 corrected in individual table. Separatins now included manually
# additional coordinates now included in individual table for my 2019 paper
# J020954.80-100223.00 from tytler seems to be a multi-arcminute sep but it's listed as 12as in their table. recomputing now


In [420]:
# just manually corrected the separations from Lena+2014; they were listed  in mas, so I shifted them by \
# 1.e-3 to be arcseconds

# manually adjusted the separation fo rthe sudou+2003 target. A wierd angular sep was listed. \
# grabbed the physical sep in cm fom paper, converted to pc, and put that into kpc for the table. Will convert from \
# kpc back to as here up above when we need to again
#********''

# SDSS J1201+30 manually added the 0.6mpc physical sep and will convert to angular above

# sep of 0.000015 manually added for Kharb 2019

# gamma ray added to qp for zhou2018

# 3C 279

#1823+568 just added in the physical seps measured by roland2013 manually
#
#3C 279  1823+568 manually adjusted; separations from roland2013 included. adjustments made to seelction methods
#
#1308+326 under vol'vach manually adjusted
#***** --> but I think this is an overlap target with roland???

# no sep for 3FGL J0449.4-4350 but adjusted selection methods

# sep added for PKS 0426-380 and gamma-ray added to PKS 0426-380 and 0301

# everything that had covino+ now has gamma-ray icluded i the periodicity selection

# seps added and sel revised for mrk 231

# PKS 2155-304 sel adjsusted no sep avail

# S4 0954+658 sel adjusted to quasi-periodicity and no sep avail

# S0528+134 no sep and keeping variability. might change the othet back too



# we need to come up with a flag or upper limit choice to adopt in all binary cases when we have nothing\
# Krause+2019 is one example here. All of those targets do not have separations listed. We'll need to either throw \
# a flag or adopt an upper limit

# E1821+643 flagged simply as -99 in separations
# recoil hypothesis still seems alright but binary hypothesis is bunk

# Bhatta 2018 flagged as -99 for seps but added radio imaging/radio to periodicity

# for Kharb+2015 ; Das+2018, 8pc is an upper limit. We will need to introduce a flag at some point. But I don't really believe this target anyhow

# kharb+2014 flagged as -99 in seps

# almost no seps listed for liu 2014 radio imaging
# only one we can include is the 6mas for 3C 454

#NGC 5515 (J141238.14+391836.5)
#Mrk 1469 (J121607.08+504930.0)
# both from gabanyi2013a; no sep listed so flagged as -99. No addition coords avail.

# papers by lewis, erlacoues 1994, gezari, doan, liu2016 (binaries), and du 21018 do not contain firm separations\
# just discussing in passing of some seps that are usually in conflict. We are flagging these all as -99 in sep.

# added the 37 pc estimate for pesce 2018. Need to reverse calculate the angular sep (which should be ~0.1'')

# added 0.1 pc for 4C+22.25 (Decarli 2010). This is an upper limit. We will need to institute a flag.

# added 33micro as for Ark 120 (Du+2018 ; Li+2019 ; Hu+2020) and 0.02 pc as seps. these come from Li+

# added 0.002 pc for Spikey (Hu+2020 ; Kun+2020)

# 4C +01.30,
# J0116-473,
# no seps provided by Liu+2004, so flagged as -99
# but machalski includes sep of optical nuclei for 3C 293. Corrected this in the individual table. Was 0.00880kpc \
# but now corrected to 0.880 kpc (or 880 pc)
# 3C 293 also includes a tag for dual agn candidate since the optical nuclei suggest a dual and not a binary

# for J1050+3456 (shields 2009) we have included their estimate of a 10**0.3 offset of the their recoil candidate

# no seps forn WISE J020446.13-050640.8 (assef 2013)

# no seps for kim +2018 (mrk 1018). flagged as -99

# no seps for steinhardt target 0956. flagged as -99

# no seps listed for v from liu 2003; flagging as -99

# for 3C 454.3 phys sep of 0.04213 pc comes from vol'vach 2007. This is an upper limit. li06 estimated something a little smaller

# BL LAC ang and phys seps drawn from caproni 2013 (0.17 pc)
# punched yup the selection methods for BL LAC

# all the sources looked only at by ciaramella have been adjusted in terms of select method and flagged as -99 in seps

# items from Rieger2007: sel methods are getting adjusted to match what he claims, but if it is only mentioned by him\
# sep is flagged as -99. I simply don't trust that work. 

# 0.000005 pc adopted for phys sep of Mrk 501. Comes from Bhatta and agrees with Villata99

# went through for all QPO sandrinelli targets and punched up the select methods

# seps for AO 0235+164 1 mas and is upper limit. comes from ostorero+04

# 5 mpc added for serafinelli target mrk 915. need to go and add selection method/possible sep to the \
# severgnini target that serafinelli also selected
#*******

# 3C 84 (britzen 2019) flagged as -99 for seps. added radio imaging to sel meth.

# added phys sep of 0.014 pc for 1308 (britzen 2015)

# 0735+178 flagged as -99 for seps (Britzen 2015)

# J1536+0441 --> moving this from dual candidate to dual but also keeping the bioanary candidate status. \
# But bondi and wrobel do a pretty good jb og showing this is very  likely dual but the vel offset still makes\
# it a candidate

# j0927 flagged as -99 because I'm not sure yet what we should ark as separations

# xu 2009 J1316 reflagged as -99 because I don't see a separation listed in paper

# NGC 5548 flagged as 0.0324078 pc for phys seps. This comes from peterson and is a lower limit. 
# peterson quote 10**17/sin i

# 3C 332 getting flagged as -99 for seps since this is a highly unlikely binary

# arp102b changed to -99 and -99 for seps. can't find my original sep in the papers
  
    
# upper limits....
# 4C+22.25

# we will need to go back and add velocity offsets for stuff like recoil candidates/binary candidates based on \
# velocity offset broad lines...

In [421]:
# and now adjusting the selection, analysis, and confirmation methodologies....
#Comerford+2009a
#Mrk 273 asked imanishi, but still need to ask vivian u

#Need to measure separations
######## ******* Need to email Fu about 2012 and 2011 papers
# calculating here separations



In [422]:
# here we are further formatting the coordinates columns
# we'll be flagging RA2 and Dec2 as -99 (value not string) for cases where RA2 and Dec2 are duplicated from RA1 and Dec1

## we'll start by flagging groups of objects based on the paper they come from:
#papers = ['Orosz+2013','Hwang+2020']
#
#for i in papers:
#    for index, row in MAC[MAC['Paper(s)']==str(i)].iterrows():
#        MAC.at[index, 'RA2'] = -99
#        MAC.at[index, 'Dec2'] = -99
#
## and here we are formatting for the naming convention for Name2
## if the entry for Name 2 is duplicated from Name1, it is reflagged as -99. Here I have gone through and manually \
## checked this
#
#papers = ['Kim+2020','Barrows+2013','Ge+2012','Lyu+2016 ; Yuan+2016','Lyu+2016','Yuan+2016','Wang+2009 ; Ge+2012',\
#         'Liu+2010a ; Ge+2012','Ge+2012 ; Orosz+2013','Smith+2010 ; Song+2020','Shi+2014',\
#         'Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020','Smith+2010 ; Kim+2020 ; Song+2020',\
#         'Smith+2010 ; Smith+2012 ; Song+2020','Smith+2010 ; Ge+2012 ; Song+2020','Wang+2009 ; Tingay+2011 ; Ge+2012',\
#         'Wang+2009 ; Liu+2010a ; Ge+2012','Liu+2010a ; Yuan+2016','Yuan+2016 ; Yang+2019 ; Joshi+2019',\
#         'Kim+2020 ; Liu+2014','Liu+2010a ; Yuan+2016','Shi+2014 ; Lyu+2016','Smith+2010 ; Yuan+2016 ; Song+2020',\
#         'Smith+2010 ; Shi+2014 ; Song+2020','Smith+2010 ; Comerford+2018 ; Song+2020',\
#         'Liu+2010a ; Ge+2012 ; Yuan+2016','Liu+2010a ; Yuan+2016 ; Comerford+2018','Ge+2012 ; Yuan+2016',\
#         'Wang+2009 ; Shi+2014','Liu+2010a ; Ge+2012 ; Comerford+2018','Liu+2010a','Ge+2012 ; Kim+2020']
##J143359.71+351020.5 (comerford2013)
#for i in papers:
#    for index, row in MAC[MAC['Paper(s)']==str(i)].iterrows():
#        MAC.at[index, 'Name2'] = "-99"

# just added A and B designations to the Miller+2004 targets, since previously they were listed as the same thing

# and evidently the objects within the gimli table have already been taken care of, so we only need to be \
# concerned with the gandalf targets


In [423]:
# and here we're going to fix up the duplicate Name1 and Name2s, duplicate coordinates between RA1 and RA2, etc...
name2fix = ['4C+22.25 / J1000+2233','084047.58+131223.5 / 3C 207','Abell 1145 / B1059+169 (Abell 1145)',\
            '3C 136.1 / 3C136.1','3C 315 / 3C315','3C 403 / 3C403','3C 52 / 3C52','4C 01.30 / 4C +01.30',\
            '4C 12.03 / 4C12.03','4C 48.29 / 4C48.29']

for index, row in MAC.iterrows():
    if (row['RA1'] == row['RA2']) and (row['Dec1'] == row['Dec2']):
        #gimli.at[index, 'Confidence Flag'] = -1
        MAC.at[index, 'RA2'] = -99
        MAC.at[index, 'Dec2'] = -99
        MAC.at[index, 'Equinox2'] = -99
        MAC.at[index, 'Coordinate_waveband2'] = -99
        MAC.at[index, 'Coordinate_Source2'] = -99
        
for index, row in MAC.iterrows():
    if row['Name1'] == row['Name2']:
        MAC.at[index, 'Name2'] = '-99'
        
for index, row in MAC.iterrows():
    if row['Name1'] == '3C 433 / 3C433':    
        MAC.at[index, 'Name2'] = '-99'
        MAC.at[index, 'RA2'] = -99
        MAC.at[index, 'Dec2'] = -99
        MAC.at[index, 'Equinox2'] = -99
        MAC.at[index, 'Coordinate_waveband2'] = -99
        MAC.at[index, 'Coordinate_Source2'] = -99
        MAC.at[index, 'Name1'] = '3C 433'
    elif row['Literature Name'] == 'J093201.60+031858.7':    
        MAC.at[index, 'Name2'] = '-99'
    elif row['Name1'] in name2fix:    
        MAC.at[index, 'Name2'] = '-99'

#for index, row in gimli.iterrows():
#    if (row['RA1'] == row['RA2']) and (row['Dec1'] == row['Dec2']):
#        #gimli.at[index, 'Confidence Flag'] = -1
#        gimli.at[index, 'RA2'] = -99
#        gimli.at[index, 'Dec2'] = -99
#        gimli.at[index, 'Equinox2'] = -99
#        gimli.at[index, 'Coordinate_waveband2'] = -99
#        gimli.at[index, 'Coordinate_Source2'] = -99
#    elif row['Name1'] == row['Name2']:
#        gimli.at[index, 'Name2'] = '-99'
#    elif row['Name1'] == '3C 433 / 3C433':    
#        gimli.at[index, 'Name2'] = '-99'
#        gimli.at[index, 'RA2'] = -99
#        gimli.at[index, 'Dec2'] = -99
#    elif row['Literature Name'] == 'J093201.60+031858.7':    
#        gimli.at[index, 'Name2'] = '-99'
#    elif row['Name1'] in name2fix:    
#        gimli.at[index, 'Name2'] = '-99'


In [424]:
#gandalfcheck = gandalf[(gandalf['RA1'].astype(str)==gandalf['RA2'].astype(str)) & (gandalf['Dec1'].astype(str)==gandalf['Dec2'].astype(str))]
#gandalfcheck

MACcheck = MAC[(MAC['RA1'].astype(str)==MAC['RA2'].astype(str)) & (MAC['Dec1'].astype(str)==MAC['Dec2'].astype(str))]

MACcheck

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag


In [ ]:
MACcheck = MAC[(MAC['Name1']==MAC['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
print(len(MACcheck))
MACcheck

In [ ]:
MACcheck = MAC[MAC['Processed System Type']=='Dual AGN / Dual AGN Candidate']
MACcheck


In [ ]:
#gandalfcheck = gandalf[(gandalf['Name1']==gandalf['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
#print(len(gandalfcheck))
#gandalfcheck

In [ ]:
#gandalfcheck = gandalf[(gandalf['Name1']==gandalf['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
#print(len(gandalfcheck))
#gandalfcheck



In [ ]:
#gimlicheck = gimli[(gimli['Name1']==gimli['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
#print(len(gimlicheck))
#gimlicheck

# did this check; no objects that have matching Name1 and Name2

In [425]:
# now I'm corrected some issues with spec/phot not being listed under the redshifts:

# For Inada+2008, these objects all have spec-zs for z1 and z2
pairs = ['SDSSJ084710.40-001302.6','SDSSJ093207.15+072251.3','SDSSJ100859.55+035104.4','SDSSJ112012.11+671116.0',\
         'SDSSJ121244.33+091208.1','SDSSJ004757.25+144741.9','SDSSJ074013.44+292648.4','SDSSJ082046.24+035742.1',\
         'SDSSJ083557.50+341455.4','SDSSJ083649.55+484154.0','SDSSJ090955.54+580143.2','SDSSJ092024.21+030636.0',\
         'SDSSJ094309.66+103400.6','SDSSJ094510.75+472448.8','SDSSJ095711.08+640548.6','SDSSJ100034.17+540628.6',\
         'SDSSJ103519.36+075258.0','SDSSJ104213.61+061942.0','SDSSJ104658.02+471726.9','SDSSJ110932.13+531635.7',\
         'SDSSJ114546.22+032251.9','SDSSJ121002.47+495312.7','SDSSJ121636.02+543159.2','SDSSJ121647.22+495720.4',\
         'SDSSJ125422.00+610421.6','SDSSJ133945.37+000946.1','SDSSJ140016.87+542131.7','SDSSJ142359.48+545250.8',\
         'SDSSJ143433.45+613752.7','SDSSJ160547.59+511330.2','SDSSJ162902.59+372430.8','SDSSJ204113.41-060158.5',\
         'SDSSJ211102.60+105038.3','SDSSJ211230.33-063332.1','RXJ1629+3724A','SDSS J1120+6711A']

# RXJ1629+3724A was listed by Mason+2000 and optical spectroscopy from Inada+20008 confirmed
# SDSS J1120+6711A was listed by Pindor+ and confirmed with Inada+

for index, row in MAC.iterrows():
    if row['Name1'] in pairs:
        MAC.at[index, 'z1_type'] = 'spec'
        MAC.at[index, 'z2_type'] = 'spec'


In [426]:
gandalf = MAC[MAC['Processed System Type'].str.contains('Dual AGN', na=False)]

unique_combinations = gandalf['Processed System Type'].dropna().unique()

print(unique_combinations)
print(len(unique_combinations))


['Dual AGN Candidate' 'Dual AGN / Dual AGN Candidate'
 'Dual AGN Candidate / Dual SMBH Candidate'
 'Dual AGN Candidate / Recoil Candidate' 'Dual AGN'
 'Binary AGN Candidate / Dual AGN Candidate'
 'Binary SMBH Candidate / Dual AGN Candidate'
 'Binary AGN Candidate / Binary SMBH Candidate / Dual AGN Candidate'
 'Binary AGN Candidate / Dual AGN Candidate / Recoil Candidate'
 'Binary AGN Candidate / Binary SMBH Candidate / Dual AGN Candidate / Recoil Candidate'
 'Binary SMBH Candidate / Dual AGN'
 'Binary SMBH Candidate / Dual AGN Candidate / Recoil Candidate']
12


In [ ]:
# and here we're going to fix up the duplicate Name1 and Name2s, duplicate coordinates between RA1 and RA2, etc...

#for index, row in gandalf.iterrows():
#    if row['Paper(s)'] == 'Hwang+2020':
#        #gimli.at[index, 'Confidence Flag'] = -1
#        gandalf.at[index, 'RA2'] = -99
#        gandalf.at[index, 'Dec2'] = -99
#        gandalf.at[index, 'Equinox2'] = -99
#        gandalf.at[index, 'Coordinate_waveband2'] = -99
#        gandalf.at[index, 'Coordinate_Source2'] = -99
#    elif row['Paper(s)'] == 'Orosz+2013':
#        gandalf.at[index, 'RA2'] = -99
#        gandalf.at[index, 'Dec2'] = -99
#        gandalf.at[index, 'Equinox2'] = -99
#        gandalf.at[index, 'Coordinate_waveband2'] = -99
#        gandalf.at[index, 'Coordinate_Source2'] = -99
#    elif row['Paper(s)'] == 'Comerford+2013':
#        gandalf.at[index, 'RA2'] = -99
#        gandalf.at[index, 'Dec2'] = -99
#        gandalf.at[index, 'Equinox2'] = -99
#        gandalf.at[index, 'Coordinate_waveband2'] = -99
#        gandalf.at[index, 'Coordinate_Source2'] = -99
#    #elif row['Paper(s)'] == 'Gattano+2014':
#    #    gandalf.at[index, 'RA2'] = -99
#    #    gandalf.at[index, 'Dec2'] = -99
#    #    gandalf.at[index, 'Equinox2'] = -99
#    #    gandalf.at[index, 'Coordinate_waveband2'] = -99
#    #    gandalf.at[index, 'Coordinate_Source2'] = -99

# nmeed to check pindor+hiennawi


# and below here, we'll start cleaning up the selection, analysis, and confirmation methdologies



In [ ]:
#aragorn = gandalf[(gandalf['RA1']==gandalf['RA2']) & gandalf['Dec1']==gandalf['Dec2']]
#
#boromir = gandalf[(gandalf['RA1']==gandalf['RA2'])]
#
#legolas = gandalf[gandalf['RA1']!=gandalf['RA2']]
#
#print(len(aragorn),len(legolas),len(boromir))



In [ ]:
# Function to check the condition
#def check_value(value):
#    if pd.isna(value) or (value != -99 and not value > 0):
#        return True
#    return False

# Applying the check and creating a new DataFrame
#filtered_df = gandalf[gandalf['z1'].apply(check_value)]

#print("Original DataFrame:")
#print(df)
#print("\nFiltered DataFrame:")
#filtered_df

# As of December 14/15th 2023, we have fixed all issues of NaN values in the redshift z1 and z2 column for duals


In [427]:
gandalf_grey = gandalf[((gandalf['z1']>0) & (gandalf['z2']<0))|((gandalf['z1']<0) & (gandalf['z2']>0))]
gandalf_white = gandalf[(gandalf['z1']>0) & (gandalf['z2']>0)]
saruman = gandalf[(gandalf['z1']<0) & (gandalf['z2']<0)]


In [ ]:
# Now down below here we're going to start working on formatting the separations and ensuring we have \
# angular separations where needed/applicable and that we don't overwrite physical separations by accident

gg = gandalf_white[(gandalf_white['Sep(kpc)']>0) & (gandalf_white['Sep']<0)]
len(gg)
#gg

In [ ]:
gg = gandalf_grey[(gandalf_grey['Sep(kpc)']>0) & (gandalf_grey['Sep']<0)]
len(gg)
gg

In [ ]:
gg = gandalf_grey[(gandalf_grey['Sep']<0)]
len(gg)
#gg

In [ ]:
gg = gimli[(gimli['Sep(kpc)']>0) & (gimli['Sep']<0)]
len(gg)
gg

In [ ]:
#for index, row in MAC.iterrows():
#    if row['z1']!=
MAC[~MAC["z2"].apply(np.isreal)]  
    

In [428]:
# running calculations for gandalf_white, where we have redshifts for both
gandalf_white['dV_new'] = -99

for index, row in gandalf_white.iterrows():
    gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
    gandalf_white.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    gandalf_white.at[index, 'Sep(kpc)_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)

#gandalf_white['Sep(kpc)'] = gandalf_white['Sep']*cosmo.kpc_proper_per_arcmin(gandalf_white['z1'])*(u.arcmin/u.kpc)*(1/60)

# now to cull the stuff we're not interested in...

bad = gandalf_white[(np.abs(gandalf_white['dV_new'])>2000)] 
bad2 = gandalf_white[((gandalf_white['Sep(kpc)_z1']>110) & (gandalf_white['Sep(kpc)_z2']>110))]
print(len(bad),len(bad2))


/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/3510151646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gandalf_white['dV_new'] = -99
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/3510151646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gandalf_white.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/3510151646.py:6: SettingWit

153 327


In [429]:
# running calculations for gandalf_grey, where we have redshifts for both
gandalf_grey['dV_new'] = -99

for index, row in gandalf_grey.iterrows():
    #gandalf_grey.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
    gandalf_grey.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)

#gandalf_grey['Sep(kpc)'] = gandalf_grey['Sep']*cosmo.kpc_proper_per_arcmin(gandalf_grey['z1'])*(u.arcmin/u.kpc)*(1/60)

# now to cull the stuff we're not interested in...

#bad3 = gandalf_grey[(np.abs(gandalf_grey['dV_new'])>2000)] 
bad3 = gandalf_grey[(gandalf_grey['Sep(kpc)_z1']>110)]
print(len(bad3))

#<0.4 is a double peaked object

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/2733027850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gandalf_grey['dV_new'] = -99
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/2733027850.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gandalf_grey.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
/Users/ryan/anaconda3/lib/python3.9/site-packages/astropy/cosmology/flrw/lambdacdm.py:404: RuntimeWarning: invali

50


In [430]:
# combining these tables back together to format the confidence flags without having the contaminants...
gandalf_white = gandalf_white[~gandalf_white['Name1'].isin(bad['Name1'])]
gandalf_white = gandalf_white[~gandalf_white['Name1'].isin(bad2['Name1'])]

gandalf_grey = gandalf_grey[~gandalf_grey['Name1'].isin(bad3['Name1'])]

gandalf = pd.concat([gandalf_white,gandalf_grey,saruman])

# and we also need to remove the bad objects from the full MAC table since we use the intersectino of the MAC \
# table with the gandalf table to generate the gimli table

MAC = MAC[~MAC['Name1'].isin(bad['Name1'])]
MAC = MAC[~MAC['Name1'].isin(bad2['Name1'])]
MAC = MAC[~MAC['Name1'].isin(bad3['Name1'])]


# here we're going to run the same calculations but run a few if else statements to make sure we're not \
# overwriting things unnecessarily
MAC['dV_new'] = -99
#MAC.at[index, 'Sep(kpc)_z1'] = -99
#MAC.at[index, 'Sep(kpc)_z2'] = -99

#gandalf_grey = gandalf[((gandalf['z1']>0) & (gandalf['z2']<0))|((gandalf['z1']<0) & (gandalf['z2']>0))]
#gandalf_white = gandalf[(gandalf['z1']>0) & (gandalf['z2']>0)]
#saruman = gandalf[(gandalf['z1']<0) & (gandalf['z2']<0)]

for index, row in gandalf_white.iterrows():
    if ((row['z1']<0) | (row['z2']<0)) & (row['dV']!=-99):
        MAC.at[index, 'dV_new'] = row['dV']
    elif (row['z1']>0) and (row['z2']>0):
        MAC.at[index, 'dV_new'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
        #MAC.at[index, 'Sep(kpc)_z1'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
        #MAC.at[index, 'Sep(kpc)_z2'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)
        MAC.at[index, 'Sep(kpc)'] = max(row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60), row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60))
    elif (row['z1']>0) and (row['z2']<0):
        MAC.at[index, 'Sep(kpc)'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
    elif (row['z1']<0) and (row['z2']>0):
        MAC.at[index, 'Sep(kpc)'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z2'])*(u.arcmin/u.kpc)*(1/60)


#gandalf_grey = gandalf[(gandalf['z1']>0) & (gandalf['z2']<0)]
#gandalf_white = gandalf[(gandalf['z1']>0) & (gandalf['z2']>0)]
#saruman = gandalf[(gandalf['z1']<0) & (gandalf['z2']<0)]


# and generating the gimli table here
gimli = MAC[~MAC['Name1'].isin(gandalf['Name1'])]
print(len(MAC),len(gandalf),len(gimli))



5683 3956 1727


In [374]:
MAC[MAC['Sep']==0]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new


In [ ]:
#MAC[((MAC['z1']<0) & (MAC['z2']<0)) & (MAC['dV']!=-99)]
MACcheck = MAC[MAC['Confidence Flag']==1]
len(MACcheck)

In [ ]:
MACcheck

In [ ]:
#bad

In [ ]:
MACcheck[MACcheck['dV']>2000]


In [ ]:
#bad2

In [ ]:
#bad3

In [ ]:
len(gandalf)

In [456]:
gandalf['Confidence Flag'] = -99

In [494]:
# now adding in the subjective flag for all of the objects in the gimli table (which are by and large binary \
# and recoil candidates, with some dual AGN and SMBH candidates added in)

# we're going to use this list of papers to apply flags
papers = ['Hwang+2020','Orosz+2013','Lyu+2016','Yuan+2016','Barrows+2013','Ge+2012','Lyu+2016 ; Yuan+2016',\
         'Wang+2009 ; Liu+2010a ; Ge+2012','Smith+2010 ; Song+2020','Shi+2014','Wang+2009 ; Ge+2012 ; Shi+2014',\
          'Liu+2010a ; Ge+2012','Liu+2010a ; Ge+2012','Liu+2010a','Kim+2020','Spiniello+2018','Rusu+2019',\
          'Wang+2009 ; Ge+2012','Wang+2009','Liu+2010a ; Yuan+2016','Liu+2010a ; Ge+2012 ; Yuan+2016',\
          'Ge+2012 ; Orosz+2013','Wang+2009 ; Shi+2014','Ge+2012 ; Comerford+2013','Smith+2010 ; Smith+2012 ; Song+2020',\
          'Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020','Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020 ',\
          'Smith+2010 ; Ge+2012 ; Song+2020 ','Yuan+2016 ; Yang+2019 ; Joshi+2019','Kim+2020 ; Liu+2014',\
          'Kim+2020 ; Kim+2016','Spiniello+2018 ; Rusu+2019',\
          'Yuan+2016 ; Cheung+2007 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018',]

for i in papers:
    for index, row in gandalf.iterrows():
        if row['Paper(s)'] == str(i):
            gandalf.at[index, 'Confidence Flag'] = 0
            
for index, row in gandalf.iterrows():       
    if row['Paper(s)'] == 'Lemon+2018':
        gandalf.at[index, 'Confidence Flag'] = +0.5  
    elif row['Paper(s)'] == 'Lemon+2019':
        gandalf.at[index, 'Confidence Flag'] = +0.5                                           
    elif row['Paper(s)'] == 'Lemon+2020':
        gandalf.at[index, 'Confidence Flag'] = +0.5   
    #elif row['Paper(s)'] == 'Findlay+2018':
    #    gandalf.at[index, 'Confidence Flag'] = +1 
    elif row['Paper(s)'] == 'Koss+2012':
        gandalf.at[index, 'Confidence Flag'] = +1  
    #elif 'Hennawi+2010' in row['Paper(s)']:
    #    gandalf.at[index, 'Confidence Flag'] = +1 
    #elif 'Hennawi+2006' in row['Paper(s)']:
    #    gandalf.at[index, 'Confidence Flag'] = +1 
    elif 'Smith+2010 ; Song+2020' in row['Paper(s)']:
        gandalf.at[index, 'Confidence Flag'] = 0 
    elif row['Paper(s)'] == 'Liu+2011b ; Ge+2012':
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif row['Paper(s)']=='Komossa+2008 ; Shields+2009 ; Bogdanović+2009 ; Dotti+2009 ; Heckman+2009 ; Decarli+2009 ; Decarli+2010 ; Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020': 
        gandalf.at[index, 'Confidence Flag'] = 0 # updated on 24 jan 2024
    elif row['Paper(s)'] == 'Fu+2018':
        gandalf.at[index, 'Confidence Flag'] = +0.5
    #elif ('Grade A targets from Fu+2015.' in row['Notes']) and (row['Paper(s)']=='Fu+2015'):
    #    gandalf.at[index, 'Confidence Flag'] = 0.5
    #elif ('Grade B targets from Fu+2015.' in row['Notes']) and (row['Paper(s)']=='Fu+2015'):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Fu+2015':
        gandalf.at[index, 'Confidence Flag'] = 0.5 # come back and fix this so that we differentiate between\
        # the grade A and B targets!


In [534]:
# same thing as above, but now I'm focusing on the gandalf_grey2 table 
ds = ['NGC 5278','Mrk 273 N','3C 321','WISE 2215-3056','J162345.20+080851.1','J114642.47+511029.6 / J1146+5110SW',\
     'J112659.54+294442.8 / J1126+2944NW','J110851.04+065901.4 / J1108+0659NW','J102325.57+324348.4','J115822.58+323102.2',\
     ]
# decide if you really believe J110851.04+065901.4
# fu classify as binary: J115106.69+471157.7 but they also note that confirmation is needed

for index, row in gandalf.iterrows():
    if row['Name1'] in ds:
        gandalf.at[index, 'Confidence Flag'] = 1

# same thing here, except that we're going to go with objects that deserve +0.5

dcs = ['III Zw 035 NE','J1221+1137 NE','J1301+2918 NE','J1536+0441 VLA-A',\
      'COSMOS J100043.15+020637.2 / CID-42','J132323.33-015941.9','VV 114 E','CXOJ1426+35','IRAS 12072-0444 N',\
      '3C 459 N1','CLASS B0827+525 A','HE0450-2958','WISE 1051-1142','PKS 1155+251 C','J115714.97+081632.0',\
      'J124859.72-025730.7','J125327.50+254747.4','J130128.77-005804.3','J132318.81+030807.1',\
      'J135024.66+240251.4','J144541.31+334107.9','J154107.81+203608.8','J155645.97+241828.5','J172049.25+310646.4',\
      'J132947.52+431357.36','J020011.52-093126.1','J072554.42+374436.9','J080337.32+392633.1',\
       'J081507.41+430427.0','J084049.47+272704.8','J090246.93+012028.2','J090615.92+121845.6',\
       'J091201.68+532036.6','J092455.24+051052.0 / J0924+0510E','J094236.68+192541.1','J105104.54+625159.3',\
       'J124813.82+362423.6','J113126.08-020459.2 / J1131-0204W','J151735.17+214532.5','J160524+152233',\
      'J115523.74+150756.9','J123915.40+531414.6 / J1239+5314NE','J132231.86+263159.1 / J1322+2631SW','J133226.34+060627.4 / J1332+0606SW',\
      'J161027.41+130806.8','J080418.23+305157.2','J123420.14+475155.9','J124037.84+353437.3',\
      'J115106.69+471157.7','J135646.11+102609.1 / J1356+1026NE']

# J115714.97+081632.0is a dual candidate bcause of the companion at ~30 kpc that cmerford found, NOT \
# because of the double peaks
# same for J124859.72-025730.7 except there are also companions within 3'' but Fu considered this to double peaked\
# object to be due to extended narrow line region
# same for J130128.77-005804.3  except fu considered it unresolved NLR
# same for J132318.81+030807.1
# same for J154107.81+203608.8
# comerford also flags J161027.41+130806.8, when Fu and others argued that the companions within 3'' has no detected emission

for index, row in gandalf.iterrows():
    if row['Name1'] in dcs:
        gandalf.at[index, 'Confidence Flag'] = 0.5
# IRAS 12072-0444 N from Imanishi+ is listed as 0.5 
# HE0450-2958 should be a slam dunk near-IR spectroscopic proposal

# and now for objects that will get a 0 flag

unclear = ['4C 40.24 (0945+408) / J094855.34+403944.6','J0841+0101 E','J0841+0101 E ','NGC 5515 (J141238.14+391836.5)',\
          'Mrk 1469 (J121607.08+504930.0)','MGB2016+112 Core 1','J1159+5320 SE','SDSS J133039.82-001035.7',\
          'J121855.80+020002.1','J162011.28+172427.5','J132450.59+175815.0','J134909.63+040448.3',\
           'J141316.25+211937.5','J142031.48+400815.9','J150102.58+394200.2','J164413.90+252828.4 ',\
           'J233604.03+000447.1','J091405.28+171554.36','J011341.11+010608.50', 'J083127.50+321926.9 / B2 0828+32',\
          'J095840.09+285239.2','J164413.90+252828.4','SDSS J1316+1753','J135251.22+654113.2','J140816.02+015528.3',\
          'J012613.31+142013.4','J080740+390015 / Mrk 622','J113721.36+612001.2','J124358.36-005845.4',\
          'J145050.60+083832.6','J085431.28-003650.6','PKS 0235+023 / J023832.67+023349.1','J101927.56+013422.5',\
          'J100921.26+013334.6','J094205.83+125433.7','J132848.46+275227.8','J145050.60+083832.6',\
           'J225252.94+002928.4','J225510.12-081234.4','J002729.24+211152.08','SDSSJ095324.39+570319.5',\
          'SDSSJ085122.37+472249.0','3C 293']
#PKS 0235+023 / J023832.67+023349.1 for the binary hypothesis it is -0.5 but for dual it is 0
# 'J085431.28-003650.6' had been put down in -0.5 for some reason
for index, row in gandalf.iterrows():
    if row['Name1'] in unclear:
        gandalf.at[index, 'Confidence Flag'] = 0
# the barrows 2013/ciamarella target isn't convincing enough to give it a 0.5
        
# and here for objects that should be flagged as -0.5
less = ['KISSR 1494','J160027.78+083743.0','J094124+394441 / 3C223.1','J121911.16+042905.9',\
       'J130724.08+460400.9','J140845.73+353218.5','J144012.74+615633.0','J151656.59+183021.5',\
       'J160631.37+273643.0','J161847.93+215925.4','J235256.62+001155.2','J140914.35+565625.7','J000656+154847',\
       'J073509+403624','J073849.75+315611.9','J074729+344018','J080315.67+483603.1','J080841.22+481351.9',\
       'J084130.18+393119.2','J085512+642345','J094032.25+311328.6','J105052.46+083934.8','J110215+290725',\
       'J152431.41+323750.6','J210449.13-000919.1','J231051.95-090011.9','J231051.95-090011.9',\
       'J233313.17+004911.8']
# J110215+290725 has a z-shaped xray source which means to me that this is prob an ism-jet iteraction
# J094032.25+311328.6 has spatially coincident emission lines within 0.15'', so they should not be originating from a companion
# J085512+642345 was looked at by comerford+ but also claimed to be a binary by severgnini and rejected by liu
for index, row in gandalf.iterrows():
    if row['Name1'] in less:
        gandalf.at[index, 'Confidence Flag'] = -0.5
# J094124+394441 / 3C223.1 being an x-shaped radio source with double peaks and no companion makes me think 
# it's def just jet-ism interactions

# and here for objects that should be flagged as -1
morebad = ['HE 0512-3329A','J151709.21+335324.7','SDSS J171544.05+600835.7','J112939.77+605742.5','J120320+131931',\
          'SDSS J1048+0055']
for index, row in gandalf.iterrows():
    if row['Name1'] in morebad:
        gandalf.at[index, 'Confidence Flag'] = -1
# J151709.21+335324.7 is the jet-ism object follow-up by Rosario

        
# figure out the problem with J095840.09+285239.2, this was apparently not in rubinur2019?
        
# and down below here, we are focusing on the double-peaked objects that have gotten a lot of attention
# we may need to come back here and fix this
#Can wholesale throw these:
#    Liu+2010a ; Yuan+2016 ; Comerford+2018 0 
#    Ge+2012 ; de Ruiter+1986 ; Merritt+2002 ; Cheung+2007 ; Merritt+2002 ; Lal+2007 0 
    

In [535]:

for index, row in gandalf.iterrows():
    if row['Notes']=='No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        #print('True')
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, No companion within 3 arcseconds. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, Fu+2011 argue the origin of the double-peaked lines is Extended narrow-line region No companion within 3 arcseconds. No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Tingay+ find no evidence of double radio cores in VLBA imaging. Fu+2011 found no companions within 3 arcseconds, No companion within 3 arcseconds. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Tingay+ find no evidence of double radio cores in VLBA imaging. No companion within 3 arcseconds. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Single structure observed in Keck AO near-IR imaging. Fu+2011 found no companions within 3 arcseconds, Fu+2011 argue the origin of the double-peaked lines is Unresolved narrow-line region No companion within 3 arcseconds. No companion within 3 arcseconds. Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Tingay+ find no evidence of double radio cores in VLBA imaging.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' No companion within 3 arcseconds. Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Single structure observed in Keck AO near-IR imaging. Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed.':
        #--> maybe a +0.5 for this, but if no companions it swings to -0.5?
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        #--> again maybe a +0.5 for this, but if no companions it swings to -0.5?
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif row['Notes']==' Single structure observed in Keck AO near-IR imaging. No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Shi+ also selected this in LAMOST. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. Comerford+2012 consider this a strong dual AGN candidate. Double optical emission line peaks observed oriented along the plane of the galaxy. Gabanyi+ do not detect a source in this system but Muller-Sanchez+ do detect two components. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        # this was a bona fide dual confirmed by muller sanchez
        gandalf.at[index, 'Confidence Flag'] = 1
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed.Muller-Sanchez+2015 find that the origin of the double-peaked emission is still ambiguous. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0 # muller sanchez claim this is still ambiguous
    elif row['Notes']==' No companion within 3 arcseconds. No companion within 3 arcseconds. Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, No companion within 3 arcseconds. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified. Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Comerford+2018 studied the kinematics that give rise to the double-peaks. Comerford+2018 found a companion within 30 kpc that also shows optical evidence of being an AGN. This is therefore a stronger case for dual AGN but not due at all to the double-peaked emission lines. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif row['Notes']==' Lyu+2016 also selected this object with BOSS. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif row['Notes']==' Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, No companion within 3 arcseconds. No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. No companion within 3 arcseconds. Comerford+2018 studied the kinematics that give rise to the double-peaks. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, No companion within 3 arcseconds. Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. Comerford+2012 consider this a strong dual AGN candidate. Double optical emission line peaks observed oriented along the plane of the galaxy. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Fu+2011 argue the origin of the double-peaked lines is Extended narrow-line region Comerford+2018 studied the kinematics that give rise to the double-peaks.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Fu+2011 found no companions within 3 arcseconds, Fu+2011 argue the origin of the double-peaked lines is Unresolved narrow-line region No companion within 3 arcseconds. No companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Fu+2011 companions within 3 arcseconds, Fu+2011 argue the origin of the double-peaked lines is Unresolved narrow-line region Companion(s) within 3 arcseconds. Companion within 3 arcseconds.Emssion line peaks show separations incompatible with the second peak originating in the companion. No continuum observed in companion or at location of second peak.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed. Comerford+2012 consider this a strong dual AGN candidate. Double optical emission line peaks observed oriented along the plane of the galaxy. No companion within 3 arcseconds.':
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif row['Notes']==' Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified.':
        gandalf.at[index, 'Confidence Flag'] = 0
    elif row['Notes']==' Companion within 3 arcseconds. Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.':
        gandalf.at[index, 'Confidence Flag'] = +0.5




    
    

In [555]:
# here, we're going to assemble a list of objects that we know should be flagged as +1, and we'll then feed \
# that list to our for loop so we can rapidly get through objects

ds = ['Mrk 266NE','FIRST J164311.3+315618A','SDSS J125455.09+084653.9 (SDSS J1254+0846 A)',\
      '1635+267A / J163700.93+263609.86','PKS 1614+051 QSO','0023+171 AB_opt','1343.4+2640A','Q1009-0252A',\
      'MGC 2214+3550A','CTQ 839A','RX J0921+4529A','SDSS J2336-0107A','NGC 6240 North','NGC 3690 E','ESO509-IG066E',\
      'Mrk 739W','4C60.07 Radio Core','J0038+4128N','X1 (Eastern)','M51a','UGC 6081',\
     'ESO 432-IG006 SW / WISEA J084427.19-314150.8','GQ 1114+1549A','SDSS J113502.03-022110.9 (QSO 1)',\
     'ELAN0101+0201 Q1','SDSS J1502+1115p','PKS 1145-071A','PHL 1222A (Q0151+048A)','Q2138-431A','2153-2056A',\
     'LBQS 0015+0239A','PKS 0347+05','J081312.63+541649.8','005113.93+002047.0','220634.96+000327.6',\
     '223222.43+001225.8','230010.19-000531.8','Mrk 463E','J0321-455N','Was 49a',' Was 49a',\
      'J113725.69+141101.3','J113725.69+141101.3','Q0142-BX195 A','J155330.23+223010.22','RXJ1629+3724A',\
      'SDSSJ084710.40-001302.6','SDSS J1120+6711A','J005113.94+002047.2','J220634.97+000327.6',\
     'J090714.45+520343.4','J154403.45+044607.5','J100602.14+071131.0','WISE 2215-3056A','IC 5338']

for index, row in gandalf.iterrows():
    if row['Name1'] in ds:
        gandalf.at[index, 'Confidence Flag'] = 1

# same thing here, except that we're going to go with objects that deserve +0.5

dcs = ['ESO 286-G17','NGC 5252','J1036+0221','J1425+3231 NW','AM1211-465NE','J1045+3519 W','J1045+3519 W ',\
       'SDSS J084810.10+351534.4','J0859+1310 NE','EGSD2 J142033.66+525917.5','UM 425A','Arp 220A','QJ0240-343A',\
       'NGC 7592 E','IRAS 14348-1447 NE','EGSD2 J141550.8+520929','NGC7674 Eastern C1','PSO J308-21 QSO',\
       'PKS 0537-441','RBS797 C1','3C 294 W','J1629+4037NW','J1107+6506N','J135225.64+142919.3 / J1352+1429W',\
       'J080529.88+241004.4','LQAC_052-000_030','LQAC_122+031_018','J2032-2358','LQAC_238+004_003',\
       'LQAC_253+026_011','J160933.21+283058.1','J031722.06+004801.8','J080523.29+281815.8','J141116.45+194436.2',\
       'J141447.15-000013.3','J085312.36+162619.5','142314.18+505537.2 / SBS 1421+511 QSO',\
      'J121345.95+024839.0 / IRAS 12112+0305 SW','J133031.75-003611.9','J105842.44+314457.6','J091543.02+300914.0',\
      'J000249.07+004504.8','J161708.89+222623.9','J1114+4036 A','J09527.62+255257.2 / J0952+2552NE',\
       'J011812.03-010442.53','022958+032031','J135429.06+132757.3','J0122+0100 NW']
# check that the redshifts for UM 425A are correct
# double check with others if they think Arp 220 is a dual based on Imanishi's work
# lemon's table lists QJ0240-343A as a binary. double check this.
# NGC7674 Eastern C1 is getting +0.5 because I think follow-up confirmation is needed 
# Teng+2012 objects getting a +0.5 because they're are larger seps and still optically selected (despite the \
# somewhat ambiguous X-ray results)
# Gattano's targets are getting 0.5 flag; eventually we'll track down if their redshifts are spec or not


for index, row in gandalf.iterrows():
    if row['Name1'] in dcs:
        gandalf.at[index, 'Confidence Flag'] = 0.5

# and now for objects that will get a 0 flag

unclear = ['IRAS 18329+5950E','J0905+3747 NE','J2356-1016 NW','Q1208+1011A']
# Lemon's catalog lists Q1208+1011A as a lens. We need to follow-up on this

for index, row in gandalf.iterrows():
    if row['Name1'] in unclear:
        gandalf.at[index, 'Confidence Flag'] = 0
        
# and here for objects that should be flagged as -0.5
less = ['FBQ 1633+3134A','J085837.53+182221.6','J094741.58+633941.4','J111519.23+542310.9','J164658.38+241132.7']
for index, row in gandalf.iterrows():
    if row['Name1'] in less:
        gandalf.at[index, 'Confidence Flag'] = -0.5
# morgan+2001 originally I was going to flag as 0 or 0.5, but given the intevening metal system, and Lemon's \
# table flagging it as a lens, I'll flag it as -0.5

# and here for objects that should be flagged as -1
morebad = ['SDSSJ132059.17+164402.59','SDSS J0818+0601A','LQAC_052-000_030','013412.78-010729.6',\
           '222051.44+005815.0'] # the latter two here are the two objects from Fu+2015 that are FR II sources
for index, row in gandalf.iterrows():
    if row['Name1'] in morebad:
        gandalf.at[index, 'Confidence Flag'] = -1


In [537]:
# adding more confidence flags:

# cleaning this up ad of 24 January 2024
papers = ['Eftekharzadeh+2017','Inada+2008','Inada+2010','Inada+2012','More+2016','Ge+2012 ; Yuan+2016',\
          'Inada+2012 ; Yang+2019','Inada+2010 ; Comerford+2014','Comerford+2013',\
          'Inada+2008 ; Graham+2015 ; Guo+2020','Inada+2008 ; Spiniello+2018',\
          'Inada+2012 ; Hwang+2020 ; Hwang+2020','Schecter+2017 ; Agnello+2018','Agnello+2018',\
          'Shi+2014 ; Lyu+2016']

for i in papers:
    for index, row in gandalf.iterrows():
        if (row['Paper(s)'] == str(i)) and ((row['z1']>0)) and ((row['z2']<0)):
            gandalf.at[index, 'Confidence Flag'] = 0
        elif (row['Paper(s)'] == str(i)) and ((row['z1']<0)) and ((row['z2']<0)):
            gandalf.at[index, 'Confidence Flag'] = 0


for index, row in gandalf.iterrows():
    #if (row['Paper(s)']=='Eftekharzadeh+2017') and ((row['z1']<0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    if ('Hennawi+2006' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])<=600):
        gandalf.at[index, 'Confidence Flag'] = 1
    elif ('Hennawi+2006' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])>600):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif ('Hennawi+2010' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])<=600):
        gandalf.at[index, 'Confidence Flag'] = 1
    elif ('Hennawi+2010' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])>600):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif ('Findlay+2018' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])<=600):
        gandalf.at[index, 'Confidence Flag'] = 1        
    elif ('Findlay+2018' in row['Paper(s)']) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')) and (np.abs(row['dV_new'])>600):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif row['Paper(s)']=='Herzog+2015':
        gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2012') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2010') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2008') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='More+2016') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)']=='Shi+2014 ; Lyu+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif (row['Paper(s)']=='Agnello+2018') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Schecter+2017 ; Agnello+2018') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2012 ; Hwang+2020 ; Hwang+2020') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2008 ; Spiniello+2018') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2008 ; Graham+2015 ; Guo+2020') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Comerford+2013') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2010 ; Comerford+2014') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif (row['Paper(s)']=='Inada+2012 ; Yang+2019') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    elif (row['Paper(s)']=='Fischer+2011') and ((row['z1']>0)) and ((row['z2']<0)):
        gandalf.at[index, 'Confidence Flag'] = -1
    elif (row['Paper(s)']=='Assef+2016') and ((row['z1']>0)) and ((row['z2']<0)):
        gandalf.at[index, 'Confidence Flag'] = -0.5
    elif (row['Paper(s)']=='Goulding+2019') and ((row['z1']>0)) and ((row['z2']<0)):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif (row['Paper(s)']=='Foord+2019') and ((row['z1']>0)) and ((row['z2']<0)):
        gandalf.at[index, 'Confidence Flag'] = -1
    #elif (row['Paper(s)']=='Ge+2012 ; Yuan+2016') and ((row['z1']>0)) and ((row['z2']<0)):
    #    gandalf.at[index, 'Confidence Flag'] = 0


papers = ['Eftekharzadeh+2017','Inada+2010 ; Gattano+2014','Myers+2007 ; Myers+2008 ; Eftekharzadeh+2017',\
         'Inada+2012 ; Eftekharzadeh+2017','Inada+2012','Inada+2010','More+2016',\
         'Myers+2007 ; Eftekharzadeh+2017','Myers+2007 ; Myers+2008 ; Inada+2010 ; Eftekharzadeh+2017',\
         'Myers+2007 ; Myers+2008 ; Inada+2012 ; Eftekharzadeh+2017','Myers+2007',\
         'Myers+2007 ; Myers+2008 ; Eftekharzadeh+2017 ; Rusu+2019','Inada+2008']

#for flag one
for i in papers:
    for index, row in gandalf.iterrows():
        if (row['Paper(s)']== str(i)) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec'))  and (np.abs(row['dV_new'])<=600):
            gandalf.at[index, 'Confidence Flag'] = 1
        elif (row['Paper(s)']== str(i)) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec'))  and (np.abs(row['dV_new'])>600):
            gandalf.at[index, 'Confidence Flag'] = 0.5
        elif (row['Paper(s)']== str(i)) and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='phot')) and (row['z2_type']=='phot'):
            gandalf.at[index, 'Confidence Flag'] = 0.5
        elif (row['Paper(s)']== str(i)) and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
            gandalf.at[index, 'Confidence Flag'] = 0

for index, row in gandalf.iterrows():
    #if (row['Paper(s)']=='Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
    #    gandalf.at[index, 'Confidence Flag'] = 1
    #elif (row['Paper(s)']=='Eftekharzadeh+2017') and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
    #    gandalf.at[index, 'Confidence Flag'] = 0
    if (row['Paper(s)']=='Eftekharzadeh+2017 ; Lemon+2020') and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif (row['Paper(s)']=='Rusu+2019 ; Lemon+2020') and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif (row['Paper(s)']=='Eftekharzadeh+2017 ; Lemon+2020 ') and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif (row['Paper(s)']=='Rusu+2019 ; Lemon+2020 ') and ((row['z1']<0)) and ((row['z2']>0)) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    #elif (row['Paper(s)']=='Inada+2010 ; Gattano+2014') and ((row['z1']>0)) and ((row['z2']>0)):
    #    gandalf.at[index, 'Confidence Flag'] = 1 
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1
    #elif (row['Paper(s)']=='Inada+2012 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1
    #elif (row['Paper(s)']=='Inada+2012') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1        
    #elif (row['Paper(s)']=='Inada+2010') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1        
    #elif (row['Paper(s)']=='More+2016 ; Hutsemekers+2020') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1        
    #elif (row['Paper(s)']=='More+2016') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1
    elif (row['Paper(s)']=='Owen+1985 ; Hudson+2006') and ((row['z1']>0)) and ((row['z2']>0)):
        gandalf.at[index, 'Confidence Flag'] = 1        
    elif (row['Paper(s)']=='Fabbiano+2011 ; Imanishi+2014 ; Koss+2015'):
        gandalf.at[index, 'Confidence Flag'] = -1        
    elif (row['Paper(s)']=='Koss+2012 ; Hainline+2016') and ((row['z1']>0)) and (row['z2']>0):
        gandalf.at[index, 'Confidence Flag'] = 1        
    elif (row['Paper(s)']=='Winter+2008 ; Koss+2012 ; Koss+2016') and ((row['z1']>0)) and ((row['z2']>0)):
        gandalf.at[index, 'Confidence Flag'] = 1        
    elif (row['Paper(s)']=='Eckert+2017') and ((row['z1']>0)) and ((row['z2']>0)):
        gandalf.at[index, 'Confidence Flag'] = 1
    elif (row['Paper(s)']=='Piconcelli+2010 ; Imanishi+2014') and ((row['z1']>0)) and ((row['z2']>0)):
        gandalf.at[index, 'Confidence Flag'] = 1
    elif (row['Paper(s)']=='Junkkarinen+2001 ; Shields+2012') and ((row['z1']>0)) and ((row['z2']>0)):
        gandalf.at[index, 'Confidence Flag'] = 1    
    #elif (row['Paper(s)']=='Myers+2007 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='phot')) and (row['z2_type']=='phot'):
    #    gandalf.at[index, 'Confidence Flag'] = +0.5                                                                         
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Inada+2010 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='phot')) and (row['z2_type']=='phot'):
    #    gandalf.at[index, 'Confidence Flag'] = +0.5       
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Inada+2012 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='phot')) and (row['z2_type']=='phot'):
    #    gandalf.at[index, 'Confidence Flag'] = +0.5   
    #elif (row['Paper(s)']=='Myers+2007') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='phot')) and (row['z2_type']=='phot'):
    #    gandalf.at[index, 'Confidence Flag'] = +0.5 
    #elif (row['Paper(s)']=='Myers+2007 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1                                                                         
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Inada+2010 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1       
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Inada+2012 ; Eftekharzadeh+2017') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1   
    #elif (row['Paper(s)']=='Myers+2007') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1 
    #elif (row['Paper(s)']=='Myers+2007 ; Myers+2008 ; Eftekharzadeh+2017 ; Rusu+2019') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1  
    #elif (row['Paper(s)']=='Inada+2008') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and (row['z2_type']=='spec'):
    #    gandalf.at[index, 'Confidence Flag'] = 1         
#Inada+2012 ; Hwang+2020 ; Hwang+2020 --> still not working....
#Inada+2008 ; Spiniello+2018
#Inada+2008 ; Graham+2015 ; Guo+2020
#Comerford+2013
#Inada+2010 ; Comerford+2014
#Inada+2012 ; Yang+2019
#Fischer+2011
#Assef+2013
#Goulding+2019
#Foord+2019
    
    # we need to go back and fix the damn overlapping object between lemon2020 and rusu2019. For some reasoon
    # we do not defer to lemon 2020 for the redshifts?
    
    #Smith+2010 ; Smith+2012 ; Song+2020
    #Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020

In [538]:
for index, row in gandalf.iterrows():
    if (row['Paper(s)']=='Liu+2011b ; Comerford+2014') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Liu+2010a ; Ge+2012 ; Nevin+2016') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Wang+2009 ; Tingay+2011') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Satyapal+2017 ; Pfeifle+2019a') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Wang+2009 ; Ge+2012') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Koss+2012') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 1
    elif (row['Paper(s)']=='Liu+2011b ; Kharb+2019') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Wang+2009') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Fu+2018') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5
    elif (row['Paper(s)']=='Liu+2011b ; Liu+2010a') and ((row['z1']>0)) and ((row['z2']>0)) and ((row['z1_type']=='spec')) and ((row['z2_type']=='spec')):
        gandalf.at[index, 'Confidence Flag'] = 0.5    

        

In [539]:
xliu = pd.read_csv('xliu2011_formated_catalog_forrflags.csv', sep=',')

xliu_agns = xliu[((xliu['FAGN1']==0) | (xliu['FAGN1']==3) | (xliu['FAGN1']==4) | (xliu['FAGN1']==5)) & \
                 ((xliu['FAGN2']==0) | (xliu['FAGN2']==3) | (xliu['FAGN2']==4) | (xliu['FAGN2']==5)) & \
                 (xliu['Sep']>10)]

len(xliu_agns)

18

In [540]:
# These are the notes from the Liu 2011 ReadME file. We'll use the optical types to add confidence flags
#--------------------------------------------------------------------------------
#      64  I1    ---     FAGN    [0/5] Sub-population flag (1)
#--------------------------------------------------------------------------------
#Note (1): Flag for sub-populations in the parent AGN sample: 
# * "0" through "2" are for narrow-line AGNs contained in the MPA-JHU DR7 catalog, where "0" stands for Seyferts,\
# "1" for LINERs, and "2" for composites, respectively, according to the Kewley et al. (2001ApJ...556..121K) and \
# Kauffmann et al. (2003MNRAS.346.1055K) criteria for separating AGNs and composites from HII regions, and \
# the Ho et al. (1997, Cat. J/ApJS/112/315) criterion for separating Seyferts from LINERS (Figure 4). 
# * "3" for narrow-line quasars in the Reyes et al. (2008, Cat. J/AJ/136/2373) sample but not in the MPA-JHU DR7 catalog; 
# * "4" for broad-line quasars, and "5" for broad-line AGNs accordinG to the Hao et al. (2005AJ....129.1783H) selection.

# so for the optical types, pairs with the flag combinations of:
# (0|3|4|5) and (0|3|4|5) will have flags of +1

# for flag combos of:
#(1|2) and (1|2) or any combo of AGN + non-AGN
# so basically once we run through and set the flags for the true AGN optical pairs, we can set the remaindeer 
# we will use flags of +0.5

#gimli = MAC[~MAC['Name1'].isin(gandalf['Name1'])]
xliu_agns_1list = xliu_agns['Name1'].to_list()
xliu_agns_2list = xliu_agns['Name2'].to_list()

for index, row in gandalf.iterrows():
    if row['Name1'] in xliu_agns_1list:
        #print('True')
        gandalf.at[index, 'Confidence Flag'] = 1 
        gandalf.at[index, 'Confirmation Method'] = gandalf.at[index, 'Selection Method']
        gandalf.at[index, 'Primary System Type'] = 'Dual AGN'
    elif (row['Paper(s)']=='Liu+2011b') and (row['Name1'] not in xliu_agns_1list):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    elif (row['Paper(s)']=='Liu+2011b ; Barrows+2016') and (row['Name1'] not in xliu_agns_1list):
        gandalf.at[index, 'Confidence Flag'] = +0.5
    
#('2011ApJ...737..101L' in row['BibCode(s)']) and (row['Name1'] not in xliu_agns_1list):

In [383]:
len(gandalf)

3956

In [471]:
len(gandalf[gandalf['Confidence Flag']>0.5])


176

In [472]:
gandalf[gandalf['Confidence Flag']<-5]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Sep(kpc)_z1,Sep(kpc)_z2
1262,1262,Single AGNs,-99,Radio Imaging / Double Radio Sources,Radio Imaging,013412.78-010729.6,0.078900,-99,01:34:12.78,-01:07:29.6,Optical,SDSS,J2000,-100.0,-100.0,-100.0,013412.80-010724.8,0.0784,-99,01:34:12.80,-01:07:24.8,J2000,Optical,SDSS,-100.0,-100.0,-100.0,136.9,4.60,-99.0,-99.0,Fu+2015 ; Fu+2015b,2015ApJ...799...72F ; 2015ApJ...815L...6F,https://doi.org/10.1088/0004-637X/799/1/72 ; h...,Faranoff-Riley II radio galaxies located withi...,-99.0,Dual AGN Candidate,Single AGNs,Dual AGN Candidate,-99,-99,-99,-99,-99,138.599167,6.858835,6.819334
1299,1299,Single AGNs,-99,Radio Imaging / Double Radio Sources,Radio Imaging,222051.44+005815.0,0.318100,-99,22:20:51.44,+00:58:15.0,Optical,SDSS,J2000,-100.0,-100.0,-100.0,222051.66+005815.8,0.3178,-99,22:20:51.66,+00:58:15.8,J2000,Optical,SDSS,-100.0,-100.0,-100.0,63.5,2.60,-99.0,-99.0,Fu+2015 ; Fu+2015b,2015ApJ...799...72F ; 2015ApJ...815L...6F,https://doi.org/10.1088/0004-637X/799/1/72 ; h...,Faranoff-Riley II radio galaxies located withi...,-99.0,Dual AGN Candidate,Single AGNs,Dual AGN Candidate,-99,-99,-99,-99,-99,68.060244,12.054830,12.047138
2047,2047,Likely Single AGN,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,-99,J011333.06+002947.9,0.044105,spec,01:13:33.06,+00:29:47.9,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.0000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,-74.0,3.00,-99.0,-99.0,Comerford+2014 ; Allen+2015,2014ApJ...789..112C ; 2015MNRAS.451.2780A,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Allen+ uses IFS observations to show that the...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.000000,2.604904,NaN
2201,2201,Likely Single AGN,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,-99,J120401.97+012641.6,0.083534,spec,12:04:01.97,+01:26:41.6,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.0000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,66.5,3.00,-99.0,-99.0,Comerford+2014 ; Allen+2015,2014ApJ...789..112C ; 2015MNRAS.451.2780A,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Allen+ use IFS observations to show that the ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.000000,4.710461,NaN
3570,3570,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J000911.58-003654.7,0.073300,spec,00:09:11.58,-00:36:54.7,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0000,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,304.0,0.25,-99.0,-99.0,Liu+2010a ; Shen+2011 ; Comerford+2012 ; Liu+2...,2010ApJ...708..427L ; 2011ApJ...735...48S ; 20...,https://doi.org/10.1088/0004-637X/708/1/427 ; ...,Shen+2011 find only a single resolved nucleu...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.000000,0.348572,NaN
3590,3590,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J011659.59-102539.1,0.150300,spec,01:16:59.59,-10:25:39.1,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0000,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,288.0,1.02,-99.0,-99.0,Liu+2010a ; Shen+2011 ; Ge+2012 ; Comerford+20...,2010ApJ...708..427L ; 2011ApJ...735...48S ; 20...,https://doi.org/10.1088/0004-637X/708/1/427 ; ...,Shen+2011 attribute the double-peaked lines ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.000000,2.671139,NaN
3601,3601,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J013546.93-005858

In [556]:
gandalfnames = gandalf['Name1'].to_list()
gandalfconf = gandalf['Confidence Flag'].to_list()

for i,j in zip(gandalfnames,gandalfconf):
    MAC.loc[MAC.Name1==str(i), 'Confidence Flag'] = j
    

In [ ]:
MACcheck = MAC[MAC['Confidence Flag']>-5]
len(MACcheck)

In [496]:
gandalf[gandalf['Confidence Flag']>0.5]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Sep(kpc)_z1,Sep(kpc)_z2
21,21,Dual AGN Candidate / Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,/ Radio Imaging,J005113.94+002047.2,0.112400,spec,00:51:13.94,+00:20:47.2,Optical,SDSS,J2000,-99.0,-99,-99,J005114.12+002049.2,0.112600,spec,00:51:14.12,+00:20:49.2,J2000,Optical,SDSS,-99.0,-99,-99,64.000000,3.360020,6.867234,-53.752809,Liu+2011b ; Fu+2015 ; Fu+2015b ; Gross+2019,2011ApJ...737..101L ; 2015ApJ...799...72F ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,Two compact steep-spectrum sources identified.,1.0,Dual AGN / Dual AGN Candidate,Dual AGN Candidate / Dual AGN,Dual AGN,-99,-99,-99,-99,-99,-53.752809,6.867234,6.877886
86,86,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,-99,J032033.21-002024.0,0.038280,spec,03:20:33.21,-00:20:24.0,Optical,SDSS,J2000,-99.0,-99,-99,J032033.65-002021.2,0.037880,spec,03:20:33.65,-00:20:21.2,J2000,Optical,SDSS,-99.0,-99,-99,115.212696,7.169273,5.443270,144.020029,Liu+2011b ; Fu+2018,2011ApJ...737..101L ; 2018ApJ...856...93F,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,115.212696,5.440557,5.386279
90,90,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,-99,J032345.63+000026.8,0.036100,spec,03:23:45.63,+00:00:26.8,Optical,SDSS,J2000,-99.0,-99,-99,J032346.35+000149.9,0.035700,spec,03:23:46.35,+00:01:49.9,J2000,Optical,SDSS,-99.0,-99,-99,121.000000,83.798866,60.127425,115.455155,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,115.455155,60.127425,59.489634
101,101,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,-99,J073905.83+442410.4,0.134370,spec,07:39:05.83,+44:24:10.4,Optical,SDSS,J2000,-99.0,-99,-99,J073906.17+442410.0,0.134250,spec,07:39:06.17,+44:24:10.0,J2000,Optical,SDSS,-99.0,-99,-99,-31.631564,3.665525,8.721166,-105.439475,Liu+2011b ; Fu+2018,2011ApJ...737..101L ; 2018ApJ...856...93F,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,31.631564,8.736073,8.729449
153,153,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,-99,J082752.27+191405.8,0.099100,spec,08:27:52.27,+19:14:05.8,Optical,SDSS,J2000,-99.0,-99,-99,J082754.96+191332.3,0.099800,spec,08:27:54.96,+19:13:32.3,J2000,Optical,SDSS,-99.0,-99,-99,179.000000,50.732147,92.818807,-190.367892,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,-190.367892,92.818807,93.399355
186,186,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,-99,J084823.91+143338.4,0.060400,spec,08:48:23.91,+14:33:38.4,Optical,SDSS,J2000,-99.0,-99,-99,J084827.76+143411.8,0.061200,spec,08:48:27.76,+14:34:11.8,J2000,Optical,SDSS,-99.0,-99,-99,223.000000,65.112981,75.948124,-225.490164,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,-225.490164,75.948124,76.881682
216,216,Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,X-ray Imaging / X-ray Spectroscopy / Optical S...,J090714.45+520343.4,0.059600,spec,09:07:14.45,+52:03:43.4,Optical,SDSS,J2000,-99.0,-99,-99,J090714.61+520

In [385]:
# splitting the table up again so that we can assign flags more easily...
gandalf['Notes'] = gandalf['Notes'].astype(str)

gandalf_grey = gandalf[((gandalf['z1']>0) & (gandalf['z2']<0))|((gandalf['z1']<0) & (gandalf['z2']>0))]
gandalf_white = gandalf[(gandalf['z1']>0) & (gandalf['z2']>0)]
saruman = gandalf[(gandalf['z1']<0) & (gandalf['z2']<0)]



In [386]:
# now assigning more confidence flags...

gandalf_grey2 = gandalf_grey[gandalf_grey['Confidence Flag']<-10]
gandalf_white2 = gandalf_white[gandalf_white['Confidence Flag']<-10]
saruman2 = saruman[saruman['Confidence Flag']<-10]
print(len(gandalf_grey2),len(gandalf_white2),len(saruman2))

34 2 0


In [ ]:
#8.380*12.04

In [ ]:
gandalf_white2


In [ ]:
gandalf_grey2


In [ ]:
#saruman2

In [ ]:
print(np.max(gandalf_white['Sep(kpc)_z1']))
print(np.max(gandalf_white['Sep(kpc)_z2']))
print(np.max(gandalf_white['dV_new']))

In [ ]:
print(len(gandalf),len(gandalf_grey),len(gandalf_white),len(saruman))

# okay, so the problem children in terms a of redshift 
# and fu2015 


# Orosz has partial redshift coverage. Hwang has no redshifts.
# I've added in the available redshifts for Lemon 2018, 2019, and 2020.
# Spiniello+ is simply going to be a problem. No redshifts and no separations listed. We'll have to reach out.
# Fu 2015 is lacking redshifts for most
# Eftek has a subset that do not have redshifts
# Rusu2019 is lacking redshifts for pretty much everything



In [ ]:
#gandalf_white

In [ ]:
#gandalf_grey

In [ ]:
# cell deprecated 24 January 2024; I don't think this is needed anymore
#for index, row in gimli.iterrows():
#    if 'Single AGNs' in row['Processed System Type']:
#        ##gimli.at[index, 'Confidence Flag'] = -1
#        #gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#        print('True singles')
#    elif 'Single AGN' in row['Processed System Type']:
#        ##gimli.at[index, 'Confidence Flag'] = -1
#        #gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#        print('True single')
#    elif 'Likely Single AGN' in row['Processed System Type']:
#        ##gimli.at[index, 'Confidence Flag'] = -1
#        #gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#        print('True likely single')
#    # deprecated 24 january 2024
#    #elif "J0045+41" in row['Name1']:
#    #    ##gimli.at[index, 'Confidence Flag'] = -1
#    #    print('Adjusted')
#    #    print('True')
#    elif 'Offset AGN Candidate' in row['Processed System Type']:
#        ##gimli.at[index, 'Confidence Flag'] = 0
#        #gimli.at[index, 'Processed System Type'] = 'Dual SMBH Candidate'
#        print('True off')
#

In [477]:
# carving up the table now to format individual pieces (hopefully this will make it easier to format... \
# it's been nothing but headaches so far...)

# correcting system types and adding some confidence flags here....
# deprecated on 24 january 2024; I don't think this is needed anymore!
#for index, row in gimli.iterrows():
#    if 'Single AGNs' in row['Processed System Type']:
#        #gimli.at[index, 'Confidence Flag'] = -1
#        gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#    elif 'Single AGN' in row['Processed System Type']:
#        #gimli.at[index, 'Confidence Flag'] = -1
#        gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#    elif 'Likely Single AGN' in row['Processed System Type']:
#        #gimli.at[index, 'Confidence Flag'] = -1
#        gimli.at[index, 'Processed System Type'] = 'Dual AGN Candidate'
#    elif "J0045+41" in row['Name1']:
#        #gimli.at[index, 'Confidence Flag'] = -1
#        print('Adjusted')
#    elif 'Offset AGN Candidate' in row['Processed System Type']:
#        #gimli.at[index, 'Confidence Flag'] = 0
#        gimli.at[index, 'Processed System Type'] = 'Dual SMBH Candidate'



unique_combinations = gimli['Processed System Type'].dropna().unique()

print(unique_combinations)
print(len(unique_combinations))


merry = gimli[gimli['Processed System Type']=='Binary SMBH Candidate']


### Fixed:
# AGN Recoil Candidate (Steinhardt+2012 target) --> changed to 'Recoil Candidate' in individual target list
# Binary AGN Candidate (Single AGN) --> changed to 'Binary AGn Candidate' in individual target list; this will get\
#     a flag of -1 in the catalog.
# Ejected Nucleus Candidate (this is the Keeney+2011 paper) --> changed this to recoil candidate. komossa+ \
#     refers to this work as a candidate, and Keeney+ also seem to suggest this could  be a recoil or ejected nucleus
# Binary SMBH Candidate / Dual SMBH Candidate / Recoil Candidate --> single menezes+2014 target
#    I'm okay with this classification until I check masses and compare against my binary definition.
# These are all converted back to dual AGN candidate, and a command has been added to insert a confidence flag:
#     Single AGNs --> these are the 2 single AGN objects from Fu+2015
#     Single AGN --> double-peaked selected, from varying samples
#     Likely Single AGN --> various double-peaked selection objects
## Offset AGN Candidate --> converting this to dual SMBH candidate in all cases

# Notes on flags to adjust:
#For literature name == LGGS J004527.30+413254.3, give flag of -1. Barth swiftly rejected this target.
# For the two 'Single AGNs'from Fu+2015a,b --> flag of -1 for these
# For the 'Single AGN' --> most likely flag -1 for these
# 'Likely Single AGN' --> most likely flag of -1 for these
# offset AGN candidate --> changing this to 'dual SMBH candidate' ? Since this is what Julie is trying to select \
# for. 

# Other things adjusted:
# Just finally added a redshift for NGC 1068
# Adjusted the matching stuff for the Barrows+2011 target so that a secondary z and name are not adopted
# Added 'bulge' and 'red blob' in name1 and name2 for the Markakis+ target

#merry

['Binary AGN Candidate' 'Binary AGN Candidate / Binary SMBH Candidate'
 'Dual SMBH Candidate'
 'Binary AGN Candidate / Dual SMBH Candidate / Recoil Candidate'
 'Binary AGN Candidate / Dual SMBH Candidate' 'Binary SMBH Candidate'
 'Recoil Candidate' 'Binary AGN Candidate / Recoil Candidate' 'Binary AGN'
 'Binary SMBH Candidate / Dual SMBH Candidate / Recoil Candidate'
 'Binary SMBH Candidate / Recoil Candidate']
11


In [478]:
# and below here, we'll start cleaning up the selection, analysis, and confirmation methdologies


In [479]:
# and now cleaning up the selection method column...
gimli['Selection Method'] = gimli['Selection Method'].astype(str)
types = gimli['Selection Method'].dropna().str.split(' / ')

# Step 2 and 3: Remove duplicates and alphabetize for each cell
def process_cell(cell):
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

# Step 4: Join the contents back into a single string
gimli['Processed Selection Method'] = processed_types.apply(' / '.join)

# Display the updated DataFrame
#print(gimli[['Processed Selection Method']])

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/3632438601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gimli['Selection Method'] = gimli['Selection Method'].astype(str)
/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/3632438601.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gimli['Processed Selection Method'] = processed_types.apply(' / '.join)


In [480]:
# now adding in the subjective flag

papers = ['']

# all  the 0 here
#for index, row in gimli.iterrows():
    
procmeth = ['Gamma-Ray Quasi-Periodicity','X-ray Imaging / X-ray Periodicity','Radio Imaging / Radio Periodicity',\
'Hard X-ray Periodicity','Optical Periodicity / Radio Periodicity','Optical Periodicity','Optical Periodicity',\
'Radio Periodicity','Modeling / X-Shaped Radio Source','Jet Precession / Modeling / Radio Imaging',\
            'Modeling / Optical Periodicity / Radio Quasi-Periodicity / Radio Periodicity / X-ray Periodicity',\
            'Jet Precession / Modeling / Radio Quasi-Periodicity / Radio Periodicity',\
            'Gamma-Ray Quasi-Periodicity / Modeling / Near-IR Quasi-Periodicity / Optical Quasi-Periodicity',\
            'Near-IR Quasi-Periodicity / Optical Quasi-Periodicity',\
            'Gamma-Ray Quasi-Periodicity / Optical Quasi-Periodicity',\
           'Modeling / Optical Periodicity / Radio Periodicity / Radio Quasi-Periodicity / X-ray Periodicity',\
           'Jet Precession / Modeling / Radio Periodicity / Radio Quasi-Periodicity',\
           'Modeling / Optical Periodicity / Radio Periodicity / Radio Quasi-Periodicity / X-ray Periodicity'] 


for index, row in gimli.iterrows():
    if row['Paper(s)'] == 'Graham+2015':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method'] in procmeth: # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Quasi-Periodicity': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Periodicity': #
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Jet Precession / Radio Imaging': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Variability': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Quasi-Periodicity / Variability': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Variability / Periodicity': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Jet precession / Periodicity / Variability': # 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Processed Selection Method']=='Periodicity / Variability': # 
        gimli.at[index, 'Confidence Flag'] = -0.5      
    elif row['Processed Selection Method']=='Modeling': # 
        gimli.at[index, 'Confidence Flag'] = -0.5    
    elif row['Processed Selection Method']=='Jet Precession': # 
        gimli.at[index, 'Confidence Flag'] = -0.5  
    elif row['Processed Selection Method']=='Jet Precession / Variability': # 
        gimli.at[index, 'Confidence Flag'] = -0.5 
    elif row['Processed Selection Method']=='X-Shaped Radio Source': # 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Graham+2015 ; Guo+2020':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Charisi+2016':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Charisi+2016 ; Guo+2020':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Comerford+2014':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Yang+2019':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Yang+2019 ; Joshi+2019':
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Krause+2019': # this is claimed jet precession
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['BibCode(s)'] == '2014ApJ...789..140L':
        gimli.at[index, 'Confidence Flag'] = 0    
    elif row['Paper(s)'] == 'Proctor+2011': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Cheung+2007 ; Proctor+2011 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018 ; Lal+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Cheung+2007 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Cheung+2007 ; Proctor+2011 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Cheung+2007 ; Roberts+2018': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Cheung+2007 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018 ; Lal+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Proctor+2011 ; Yang+2019 ; Joshi+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Proctor+2011 ; Yang+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Proctor+2011 ; Merritt+2002 ; Lal+2007': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)'] == 'Jonker+2010 ; Heida+2015': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)'] == 'Caldwell+2014': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)'] == 'Pesce+2018': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif 'Single AGN' in row['Legacy System Type']: 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)'] == 'Lena+2014': # these are recoil candidates based on positional offsets
        gimli.at[index, 'Confidence Flag'] = 0
    elif 'Maness+2004' in row['Paper(s)']: # this is the rodriguez+06 target, CSO 0402
        gimli.at[index, 'Confidence Flag'] = 1
    elif row['BibCode(s)'] == '2016ApJ...824..122K': # this is Kim+2016
        gimli.at[index, 'Confidence Flag'] = 0
    elif '1988ApJ...325..628S' in row['BibCode(s)']: # this is OJ 287 and Salanpää+88
        gimli.at[index, 'Confidence Flag'] = +0.5 
    elif row['Literature Name'] == 'LGGS J004527.30+413254.3': # Barth+ showed emphatically this is not a good candidate
        gimli.at[index, 'Confidence Flag'] = -1 #2017ApJ...850...86D ; 2018ApJ...859...10B
    elif row['Paper(s)']=='Keeney+2011': # this is recoil candidate/ejected nucleus
        gimli.at[index, 'Confidence Flag'] = +0.5
    # below here are the three promosing targets from E12, bumping it to +0.5, 
    #e12prom = ['093844','095036','161911']
    elif row['Name1']=='093844.45+005715.7':
        gimli.at[index, 'Confidence Flag'] = 0.5
    elif row['Name1']=='095036.75+512838.1':
        gimli.at[index, 'Confidence Flag'] = 0.5
    elif row['Name1']=='161911.24+501109.2':
        gimli.at[index, 'Confidence Flag'] = 0.5
    # and now for the remainder of their sample
    elif row['Paper(s)']=='Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Halpern+1992 ; Eracleous+1997 ; Gezari+2007 ; Liu+2016 ; Doan+2020': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Halpern+1988 ; Miller+1990 ; Halpern+1992 ; Eracleous+1997 ; Gezari+2007 ; Liu+2016 ; Doan+2020': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Peterson+1987 ; Sergeev+1997 ; Sergeev+2007 ; Li+2016; Bon+2016 ; Ilić+2017': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Marziani+1993 ; Gezari+2007 ; Liu+2016 ; Doan+2020 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Lewis+2010 ; Liu+2016 ; Doan+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5        
    elif row['Paper(s)']=='Gezari+2007 ; Krause+2019': 
        gimli.at[index, 'Confidence Flag'] = -0.5        
    elif row['Paper(s)']=='Krezinger+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5        
    elif row['Paper(s)']=='Webb+1990 ; Caproni+2004b ; Vol\'vach+2010 ; Du+2018': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Hu+2020 ; Kun+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Schmidt+1965 ; Merritt+2002 ; Cheung+2007 ; Merritt+2002 ; Lal+2007': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Schmidt+1965 ; Merritt+2002 ; Cheung+2007 ; Merritt+2002 ; Lal+2007 ; Krause+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Graham+2015a ; D\'Orazio+2015 ; Charisi+2015 ; Kun+2015 ; Jun+2015 ; Vaughan+2015 ; Mohan+2016 ; Liu+2018 ; Kovacevic+2019 ; Graham+2015 ; Guo+2020': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Sudou+2003 ; Jenet+2004 ; Iguchi+2010 ; Sudou+2017': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Blundell+2001 ; Robinson+2010 ; Shapovalova+2016 ; Ilić+2017': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Roland+2013 ; Qian+2013 ; Sandrinelli+2016 ; Qian+2019': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Conway+1995 ; Villata+1999 ; Rieger+2000 ; Rieger+2003 ; Rieger+2007 ; Bhatta+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Batcheldor+2010': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Yan+2015 ; Leighly+2016 ; Kovaˇcevi´c+2020': 
        gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Menezes+2016': 
        gimli.at[index, 'Confidence Flag'] = +0.5
    elif row['Paper(s)']=='Menezes+2014': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Tsai+2013': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Shen+2013': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Ju+2013': 
        gimli.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)']=='Yan+2015 ; Leighly+2016 ; Kovaˇcevi´c+2020': 
    #    gimli.at[index, 'Confidence Flag'] = -1
    elif row['Paper(s)']=='Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Liu+2014 ; Kim+2016': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Liu+2014 ; Krause+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Koss+2014': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Eracleous+1994 ; Lewis+2010 ; Liu+2016 ; Doan+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Hummel+1992 ; Roos+1993 ; Murphy+2003 ; Kun+2014': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 ; Ju+2013 ; Wang+2017': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Comerford+2014 ; Kharb+2020': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Comerford+2014 ; Yang+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Ju+2013 ; Wang+2017': 
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Paper(s)']=='Wang+2017': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Markakis+2015': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Ackermann+2015 ; Sobacchi+2017 ; Cavaliere+2017 ; Caproni+2017 ; Tavani+2018 ; Sandrinelli+2018 ; Munar-Adrover+2019 ; Covino+2019':
        gimli.at[index, 'Confidence Flag'] = -0.5        
    elif row['Paper(s)']=='Eracleous+1994': 
        gimli.at[index, 'Confidence Flag'] = 0       
    elif row['Paper(s)']=='Eracleous+1994 ; Krause+2019': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Doan+2020': 
        gimli.at[index, 'Confidence Flag'] = -0.5        
    elif row['Paper(s)']=='Gezari+2007 ; Liu+2016 ; Doan+2020 ; Proctor+2011': 
        gimli.at[index, 'Confidence Flag'] = 0 # proctor's work could still suggest it's a recoil or binary but optically this is not a binary        
    elif row['Paper(s)']=='Du+2018': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Comerford+2013': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Comerford+2009a': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Barrows+2016': 
        gimli.at[index, 'Confidence Flag'] = 0.5
    elif row['Paper(s)']=='Du+2018 ; Li+2019 ; Hu+2020': 
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Paper(s)']=='Ciaramella+2004 ; Rieger+2007 ; Vol\'vach+2010 ; Caproni+2013 ; Sandrinelli+2017 ; Covino+2019': 
        gimli.at[index, 'Confidence Flag'] = 0

#Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 --> 0
#Liu+2014 ; Kim+2016 --> 0
#Liu+2014 ; Krause+2019 --> 0
#Koss+2014 --> 0
#Eracleous+1994 ; Lewis+2010 ; Liu+2016 ; Doan+2020 --> -0.5
#Ackermann+2015 ; Sobacchi+2017 ; Cavaliere+2017 ; Caproni+2017 ; Tavani+2018 ; Sandrinelli+2018 ; Munar-Adrover+2019 ; Covino+2019 --> -0.5
#Hummel+1992 ; Roos+1993 ; Murphy+2003 ; Kun+2014 --> -0.5
#Markakis+2015 --> 0
#Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 ; Ju+2013 ; Wang+2017 --> -0.5
#Comerford+2014 ; Kharb+2020 --> 0
#Comerford+2014 ; Yang+2019 --> 0
#Ju+2013 ; Wang+2017 --> -0.5
#Wang+2017 --> 0
#Eracleous+1994 --> 0      
        
#Single AGN
#Lewis+2010 ; Liu+2016 ; Doan+2020 --> -0.5
#Gezari+2007 ; Krause+2019 --> -0.5
#Krezinger+2020 --> -0.5
#Menezes+2016 --> +0.5
#Menezes+2014 --> 0
#Tsai+2013 --> 0
#Hu+2020 ; Kun+2020 --> -0.5
#Webb+1990 ; Caproni+2004b ; Vol'vach+2010 ; Du+2018 --> -0.5
#Shen+2013 --> 0
#Ju+2013 --> 0
#Halpern+1992 ; Eracleous+1997 ; Gezari+2007 ; Liu+2016 ; Doan+2020 --> -1
#Halpern+1988 ; Miller+1990 ; Halpern+1992 ; Eracleous+1997 ; Gezari+2007 ; Liu+2016 ; Doan+2020 --> -1
#Peterson+1987 ; Sergeev+1997 ; Sergeev+2007 ; Li+2016; Bon+2016 ; Ilić+2017 --> -0.5
#Marziani+1993 ; Gezari+2007 ; Liu+2016 ; Doan+2020 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 --> -1
#Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 --> -0.5
#Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 --> -0.5
#Komossa+2008 ; Shields+2009 ; Bogdanović+2009 ; Dotti+2009 ; Heckman+2009 ; Decarli+2009 ; Decarli+2010 ; Tsalmantza+2011 ; Decarli+2013 ; Lusso+2014 ; Liu+2014 ; Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020 --> +0.5
#Schmidt+1965 ; Merritt+2002 ; Cheung+2007 ; Merritt+2002 ; Lal+2007 --> 0
#Schmidt+1965 ; Merritt+2002 ; Cheung+2007 ; Merritt+2002 ; Lal+2007 ; Krause+2019 --> 0
#Graham+2015a ; D'Orazio+2015 ; Charisi+2015 ; Kun+2015 ; Jun+2015 ; Vaughan+2015 ; Mohan+2016 ; Liu+2018 ; Kovacevic+2019 ; Graham+2015 ; Guo+2020
#--> -1
#Sudou+2003 ; Jenet+2004 ; Iguchi+2010 ; Sudou+2017 --> -0.5
#Blundell+2001 ; Robinson+2010 ; Shapovalova+2016 ; Ilić+2017 --> 0 # only for the recoil, the bianry is -0.5
#Roland+2013 ; Qian+2013 ; Sandrinelli+2016 ; Qian+2019 --> -0.5
#Conway+1995 ; Villata+1999 ; Rieger+2000 ; Rieger+2003 ; Rieger+2007 ; Bhatta+2019 --> 0
# not sure how i feel about this one
#Batcheldor+2010 --> 0
#Yan+2015 ; Leighly+2016 ; Kovaˇcevi´c+2020 --> -1

/var/folders/_n/4v22wftn7354wlkmn_g7rtdr0000gn/T/ipykernel_22628/671546408.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gimli.at[index, 'Confidence Flag'] = -0.5


In [ ]:
gimli1 = gimli[gimli['Confidence Flag'] < -10]
print(len(gimli1))
gimli1

In [ ]:
gim = gimli1[['Name1','Paper(s)','Notes','Processed System Type']].copy()

gim

In [481]:
dcs = ['084716.03+373218.0','085237.01+200410.9','092837.98+602521.0','103059.09+310255.7',\
      '111230.89+181311.4','4C55.19 (J100157.93+554047.8 / NGC 3079)','B2 1213 + 350 (J121555.60+344815.2)',\
      'B2 1630 + 35','4C+22.25 / J1000+2233','3C 186 HOST',' 3C 186 HOST','SDSS 0956+5128','CXO J101527.2+625911',\
      '153705.95+005522.8']
#4C55.19 (J100157.93+554047.8 / NGC 3079)
# B2 1213 + 350 (J121555.60+344815.2)
#B2 1630 + 35
# Despite the overall lack of info on the above three objects, they were selected based on radio morphology \
# so I think it makes sens to give them a +0.5 because they could easily be ruled out with new obs. \
# They're much less ambiguous
# SDSS 0956+5128 steinhardt's target is getting a +0.5
# CXO J101527.2+625911 has both a spatial and spectra offset. promising.
# 4C+22.25 / J1000+2233 had HUGE blue shifts. Seems reasonable to give it a 0.5 based on this
# guo consider these as binary candidates: 153705.95+005522.8

# these get flags of 0
unclear = ['110050.99+170934.2','082930.59+272822.7','SDSS J1201+30','B1834+620','OX 169','RXS J10304+5516',\
          'RX J1042+1212','Mrk 1018','SDSSJ0932+0318 / J0932+0318']
# being conservative with Mrk 1018
# unclear for binary on SDSSJ0932+0318 / J0932+0318 but is really a -0.5 for recoil

# these get flags of -0.5
less = ['Mrk 6','3C 454.3','3C 390.3','032213.89+005513.4','141020.57+364322.7','155053.16+052112.1',\
       '234932.77-003645.8','J1714+3327']
# guo do not consider these as candidates for sub-pc binaries:
#032213.89+005513.4
#141020.57+364322.7
#155053.16+052112.1
#234932.77-003645.8


# these get flags of -1
remove = ['J101847.57+294114.1','J105553.64+152027.5','J111729.22+614015.2','J134640.79+522836.5']


for index, row in gimli.iterrows():
    if row['Name1'] in dcs:
        gimli.at[index, 'Confidence Flag'] = +0.5
    elif row['Name1'] in unclear:
        gimli.at[index, 'Confidence Flag'] = 0
    elif row['Name1'] in less:
        gimli.at[index, 'Confidence Flag'] = -0.5
    elif row['Name1'] in remove:
        gimli.at[index, 'Confidence Flag'] = -1
        
        


In [482]:
gimlinames = gimli['Name1'].to_list()
gimliconf = gimli['Confidence Flag'].to_list()

for i,j in zip(gimlinames,gimliconf):
    MAC.loc[MAC.Name1==str(i), 'Confidence Flag'] = j

print(len(gimli))

1727


In [483]:
gimli1 = gimli[gimli['Confidence Flag'] < -5]
print(len(gimli1))
#gimli1

0


In [616]:
# and now here we're adding the confidence flag to the primary confidence flag column before preparing to add \
# secondary and tertiary confidence flags

# first we apply the original confidence flag to the primary confidence flag column. \
#That should be sufficient for the majority of the sample
#MAC['ST1 Confidence Flag'] = MAC['Confidence Flag']

# now we're going to check to see how many objects need a secondary class flag
#print(len(MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']=='-99')]))
#print(len(MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']!='-99')]))

# here we can start to assign secondary and tertiary flags
selmeth = ['X-Shaped Radio Source','Radio Imaging / X-Shaped Radio Source','X-Shaped Radio Source / Radio Imaging']

for i in selmeth:
    for index, row in MAC.iterrows():
        if (row['Selection Method']==str(i)) & (row['Secondary System Type']!='-99') & (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0
        #elif row[MAC['Selection Method']==str(i)] and (MAC['Secondary System Type']!='-99') and (MAC['Tertiary System Type']=='-99'):
        #    MAC.at[index, 'ST2 Confidence Flag'] = 0

# manually adding in flags for objects that have two object classes:

papers = ['Orosz+2013','Proctor+2011 ; Merritt+2002 ; Lal+2007','Popovic+2012','Ge+2012 ; Orosz+2013',\
          'Ciaramella+2004 ; Barrows+2013','Kim+2020 ; Liu+2014']
for i in papers:
    for index, row in MAC.iterrows():
        if (row['Paper(s)']==str(i)) & (row['Secondary System Type']!='-99'): # and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0

objs = ['J080529.88+241004.4','J140343.63+292045.3 / KISSR 434 A+B','J141116.45+194436.2',\
        'J124135.09+285036.5 / KISSR 102 N1-A','3C 433','3C 433 / 3C433','E1821+643','4C +01.30',\
        'J0116-473','J0437+2456','PKS 0537-441','3C 227 / J094745.2+072518','J080740+390015 / Mrk 622',\
        'J085431.28-003650.6','145824.46+363119.5','075819.69+421935.1','J155318.72+170202.9',\
       'J132848.46+275227.8','J154637.12+122832.5','J020011.52-093126.1','J09527.62+255257.2 / J0952+2552NE'] 
# these objects only warrant a 0 in the secondary column
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) & (row['Secondary System Type']!='-99'): # and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0    

objs = ['HE0450-2958','COSMOS J100043.15+020637.2 / CID-42'] 
# these objects only warrant a -0.5 in the secondary column
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) & (row['Secondary System Type']!='-99'): # and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0.5    

objs = ['SDSSJ081617.73+293639.6','PKS 0235+023 / J023832.67+023349.1']  
# these objects only warrant a -0.5 in the secondary column
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) & (row['Secondary System Type']!='-99'): # and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = -0.5    

objs = ['SDSS J1048+0055','J1536+0441 VLA-A'] 
# these objects only warrant a -0.5 in the secondary column
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) & (row['Secondary System Type']!='-99'): # and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = -1    
            
            
# manually adding the second and third flags for the handful of targets that have three object classes (15)

#0.5 and 0.5 for now?
objs = ['SDSS J0927+2943']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)): #and (row['Secondary System Type']!='-99') and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = -1
            MAC.at[index, 'ST3 Confidence Flag'] = -1

objs = ['3C 293','NGC 3115','J083127.50+321926.9 / B2 0828+32','J094124+394441 / 3C223.1','J114016.98+174340.4',\
        'SDSSJ152806.63+132345.8 / J1528+1323','J003636.21+004853.45','J155416.08+381132.64',\
        'J091405.28+171554.36','J131638.16+242732.40','J014719.27-085119.58','J011341.11+010608.50',\
        'J161847.93+215925.4']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)): #and (row['Secondary System Type']!='-99') and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0
            MAC.at[index, 'ST3 Confidence Flag'] = 0


objs = ['142314.18+505537.2 / SBS 1421+511 QSO']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)): #and (row['Secondary System Type']!='-99') and (row['Tertiary System Type']=='-99'):
            MAC.at[index, 'ST2 Confidence Flag'] = 0
            MAC.at[index, 'ST3 Confidence Flag'] = 0.5



573
15


In [617]:
#len(MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']=='-99')])
len(MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']=='-99') & \
        (MAC['ST2 Confidence Flag']==-99) & (MAC['ST3 Confidence Flag']==-99)])


0

In [618]:
MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']=='-99') & \
        (MAC['ST2 Confidence Flag']==-99) & (MAC['ST3 Confidence Flag']==-99)]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Analysis Method


In [573]:
#MAC[(MAC['Secondary System Type']!='-99') & (MAC['Tertiary System Type']!='-99')]['Name1'].to_list()

In [ ]:
# and here we're checking to see if there any objects in need of secondary or \
# tertiary conferidnce flags




In [557]:
# here we're going to be working on the confrimation and new analysis methods column

MAC['Analysis Method'] = MAC['Selection Method']

#xliu_agns_1list = xliu_agns['Name1'].to_list()
#xliu_agns_2list = xliu_agns['Name2'].to_list()

for index, row in MAC.iterrows():
    if row['Name1'] in xliu_agns_1list:
        #print('True')
        MAC.at[index, 'Confirmation Method'] = MAC.at[index, 'Selection Method']
        MAC.at[index, 'Primary System Type'] = 'Dual AGN'
        
for index, row in MAC.iterrows():
    if (row['Confidence Flag'] == 1) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])
    elif (row['Confidence Flag'] == 0.5) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'
    elif (row['Confidence Flag'] == 0.0) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'    
    elif (row['Confidence Flag'] == -0.5) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'   
    elif (row['Confidence Flag'] == -1) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'   

# manually adjusting the confirmation strats on a few targets
papers = ['Inada+2010 ; Gattano+2014','Inada+2012 ; Eftekharzadeh+2017','Inada+2012']
for i in papers:
    for index, row in MAC.iterrows():
        if (row['Paper(s)']==str(i)) and (row['Confidence Flag'] == 1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = str(MAC.at[index, 'Selection Method'])


# more manual adjustments to confirmation strategies:
objs = ['J100602.14+071131.0','J102325.57+324348.4','J115822.58+323102.2','J162345.20+080851.1','IC 5338']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == 1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Radio Imaging'

# and down below here we'll need to make 'Dual AGN Candidate' --> 'Dual AGN' for confidence flags of 1





In [558]:
print(len(MAC[MAC['System Type']=='Dual AGN']))
print(len(MAC[(MAC['System Type']=='Dual AGN') & (MAC['Confidence Flag']!=1)]))
print(len(MAC[MAC['Confidence Flag']==1]))

62
2
157


In [544]:
MAC[(MAC['System Type']=='Dual AGN') & (MAC['Confidence Flag']!=1)]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Analysis Method
3451,3451,Dual AGN,SDSS J132323.33-015941.9,Optical Imaging / UV Imaging,-99,J132323.33-015941.9,0.350,spec,13:23:23.33,-01:59:41.9,Optical,SDSS,J2000,-99,-99,-99,-99,-99.0,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,0.17,0.8,-99.0,Woo+2014,2014MNRAS.437...32W,https://doi.org/10.1093/mnras/stt1846,Coordinates not provided by Woo+ beyond the SD...,0.5,Dual AGN,Dual AGN,Dual AGN,-99,-99,-99,-99,-99,-99.0,Optical Imaging / UV Imaging
3521,3521,Dual AGN,IRAS 12072-0444,Near-IR Colors / Near-IR Imaging,-99,IRAS 12072-0444 N,0.128,-99,12:09:45.1187842872,-05:01:13.431047484,Optical,GAIA,J2000,-99,-99,-99,IRAS 12072-0444 S,-99.0,-99,-99,-99,-99,-99,-99,-99,-99,-99,-99.0,0.96,-99.0,-99.0,Imanishi+2020,2020ApJ...891..140I,https://doi.org/10.3847/1538-4357/ab733e,No coordinates or separations in arcseconds ar...,0.5,Dual AGN,Dual AGN,Dual AGN,-99,-99,-99,-99,-99,-99.0,Near-IR Colors / Near-IR Imaging


In [561]:
MAC[MAC['Confidence Flag']==1]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Analysis Method
21,21,Dual AGN Candidate / Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,/ Radio Imaging,J005113.94+002047.2,0.112400,spec,00:51:13.94,+00:20:47.2,Optical,SDSS,J2000,-99.0,-99,-99,J005114.12+002049.2,0.112600,spec,00:51:14.12,+00:20:49.2,J2000,Optical,SDSS,-99.0,-99,-99,64.000000,3.360020,6.877886,-53.752809,Liu+2011b ; Fu+2015 ; Fu+2015b ; Gross+2019,2011ApJ...737..101L ; 2015ApJ...799...72F ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,Two compact steep-spectrum sources identified.,1.0,Dual AGN / Dual AGN Candidate,Dual AGN Candidate / Dual AGN,Dual AGN,-99,-99,-99,-99,-99,-53.752809,Fiber Spectroscopy / Optical Spectroscopy / Op...
90,90,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Fiber Spectroscopy / Optical Spectroscopy / Op...,J032345.63+000026.8,0.036100,spec,03:23:45.63,+00:00:26.8,Optical,SDSS,J2000,-99.0,-99,-99,J032346.35+000149.9,0.035700,spec,03:23:46.35,+00:01:49.9,J2000,Optical,SDSS,-99.0,-99,-99,121.000000,83.798866,60.127425,115.455155,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN,-99,-99,-99,-99,-99,115.455155,Fiber Spectroscopy / Optical Spectroscopy / Op...
153,153,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Fiber Spectroscopy / Optical Spectroscopy / Op...,J082752.27+191405.8,0.099100,spec,08:27:52.27,+19:14:05.8,Optical,SDSS,J2000,-99.0,-99,-99,J082754.96+191332.3,0.099800,spec,08:27:54.96,+19:13:32.3,J2000,Optical,SDSS,-99.0,-99,-99,179.000000,50.732147,93.399355,-190.367892,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN,-99,-99,-99,-99,-99,-190.367892,Fiber Spectroscopy / Optical Spectroscopy / Op...
186,186,Dual AGN Candidate,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Fiber Spectroscopy / Optical Spectroscopy / Op...,J084823.91+143338.4,0.060400,spec,08:48:23.91,+14:33:38.4,Optical,SDSS,J2000,-99.0,-99,-99,J084827.76+143411.8,0.061200,spec,08:48:27.76,+14:34:11.8,J2000,Optical,SDSS,-99.0,-99,-99,223.000000,65.112981,76.881682,-225.490164,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,1.0,Dual AGN Candidate,Dual AGN Candidate,Dual AGN,-99,-99,-99,-99,-99,-225.490164,Fiber Spectroscopy / Optical Spectroscopy / Op...
216,216,Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,X-ray Imaging / X-ray Spectroscopy / Optical S...,J090714.45+520343.4,0.059600,spec,09:07:14.45,+52:03:43.4,Optical,SDSS,J2000,-99.0,-99,-99,J090714.61+520350.7,0.060200,spec,09:07:14.61,+52:03:50.7,J2000,Optical,SDSS,-99.0,-99,-99,195.000000,7.447625,8.660222,-169.261238,Liu+2011b ; Barrows+2016 ; Hou+2019,2011ApJ...737..101L ; 2016ApJ...829...37B ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,Selected by Barrows+ as an offset AGN candid...,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,-99,-99,-99,-169.261238,Fiber Spectroscopy / Optical Spectroscopy / Op...
284,284,Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Fiber Spectroscopy / Optical Spectroscopy / Op...,J094554.41+423840.0,0.074500,spec,09:45:54.41,+42:38:40.0,Optical,SDSS,J2000,-99.0,-99,-99,J094554.49+423818.7,0.074800,spec,09:45:54.49,+42:38:18.7,J2000,Optical,SDSS,-99.0,-99,-99,90.000000,21.318283,30.279062,-83.469035,Liu+2011b ; De Rosa+2018 ; De Rosa+2018,2011ApJ...737..101L ; 2018MNRAS.480.1

In [559]:
print(len(MAC[MAC['Confirmation Method']!='-99']))
print(len(MAC[(MAC['Confirmation Method']!='-99') & (MAC['Confidence Flag']==-99)]))
print(len(MAC[(MAC['Confirmation Method']!='-99') & (MAC['Confidence Flag']==1)]))
print(len(MAC[(MAC['Confirmation Method']=='-99') & (MAC['Confidence Flag']==1)]))


157
0
157
0


In [554]:
len(MAC[(MAC['Confirmation Method']=='-99') & (MAC['Confidence Flag']==-99)])


34

In [565]:
MAC[MAC['Confidence Flag']==-99]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Analysis Method
2047,2047,Likely Single AGN,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,-99,J011333.06+002947.9,0.044105,spec,01:13:33.06,+00:29:47.9,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,-74.0,3.00,-99.0,-99.0,Comerford+2014 ; Allen+2015,2014ApJ...789..112C ; 2015MNRAS.451.2780A,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Allen+ uses IFS observations to show that the...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.0,Optical Spectroscopy / Fiber Spectroscopy / Ve...
2201,2201,Likely Single AGN,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,-99,J120401.97+012641.6,0.083534,spec,12:04:01.97,+01:26:41.6,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,66.5,3.00,-99.0,-99.0,Comerford+2014 ; Allen+2015,2014ApJ...789..112C ; 2015MNRAS.451.2780A,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Allen+ use IFS observations to show that the ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.0,Optical Spectroscopy / Fiber Spectroscopy / Ve...
3570,3570,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J000911.58-003654.7,0.073300,spec,00:09:11.58,-00:36:54.7,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,304.0,0.25,-99.0,-99.0,Liu+2010a ; Shen+2011 ; Comerford+2012 ; Liu+2...,2010ApJ...708..427L ; 2011ApJ...735...48S ; 20...,https://doi.org/10.1088/0004-637X/708/1/427 ; ...,Shen+2011 find only a single resolved nucleu...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.0,Double-Peaked Optical Spectroscopic Emission L...
3590,3590,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J011659.59-102539.1,0.150300,spec,01:16:59.59,-10:25:39.1,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,288.0,1.02,-99.0,-99.0,Liu+2010a ; Shen+2011 ; Ge+2012 ; Comerford+20...,2010ApJ...708..427L ; 2011ApJ...735...48S ; 20...,https://doi.org/10.1088/0004-637X/708/1/427 ; ...,Shen+2011 attribute the double-peaked lines ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.0,Double-Peaked Optical Spectroscopic Emission L...
3601,3601,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J013546.93-005858.5,0.159500,spec,01:35:46.93,-00:58:58.5,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,314.0,0.29,-99.0,-99.0,Liu+2010a ; Shen+2011 ; Ge+2012 ; Comerford+20...,2010ApJ...708..427L ; 2011ApJ...735...48S ; 20...,https://doi.org/10.1088/0004-637X/708/1/427 ; ...,Shen+2011 attribute the double-peaked lines ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN Candidate,-99,-99,-99,-99,-99,-99.0,Double-Peaked Optical Spectroscopic Emission L...
3602,3602,Likely Single AGN,-99,Double-Peaked Optical Spectroscopic Emission L...,-99,J013555.82+143529.7,0.071900,spec,01:35:55.82,+14:35:29.7,Optical,-99,J2000,-100.0,-100.0,-100.0,-99,-99.0,-99,-99,-99,-99,-99,-99,-100.0,-100.0,-100.0,266.0,0.82,-99.0,-99.0,Wang+2009 ; Liu+2010a ; Shen+2011 ; Ge+2012 ; ...,2009ApJ...705L..76W ; 2010ApJ...708..427L ; 20...,https://doi.org/10.1088/0004-637X/705/1/L76 ; ...,Shen+2011 attribute the double-peaked lines ...,-99.0,Dual AGN Candidate,Likely Single AGN,Dual AGN

In [562]:
print(len(MAC[MAC['Confidence Flag']==-1]))

22


In [564]:
# making manual adjustments to confirmation strategies for objects that we've flagged as -1:
objs = ['J101847.57+294114.1','J105553.64+152027.5','J111729.22+614015.2','J134640.79+522836.5',\
        '013412.78-010729.6','222051.44+005815.0','Mrk 78','J112939.77+605742.5']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == -1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Radio Imaging'

objs = ['SDSS J0914+0853']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == -1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy'

objs = ['SDSS J0818+0601A']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == 1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Optical Imaging / Optical Spectroscopy'



#MAC[MAC['Confidence Flag']==-1]


# need tocome back and fin


,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,Primary System Type,Secondary System Type,Tertiary System Type,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Analysis Method
1262,1262,Single AGNs,-99,Radio Imaging / Double Radio Sources,Radio Imaging,013412.78-010729.6,0.078900,-99,01:34:12.78,-01:07:29.6,Optical,SDSS,J2000,-100.0,-100.0,-100.0,013412.80-010724.8,0.078400,-99,01:34:12.80,-01:07:24.8,J2000,Optical,SDSS,-100.0,-100.0,-100.0,136.900000,4.600000,6.858835,-99.0,Fu+2015 ; Fu+2015b,2015ApJ...799...72F ; 2015ApJ...815L...6F,https://doi.org/10.1088/0004-637X/799/1/72 ; h...,Faranoff-Riley II radio galaxies located withi...,-1.0,Dual AGN Candidate,Single AGNs,Dual AGN Candidate,-99,-99,-99,-99,-99,138.599167,Radio Imaging / Double Radio Sources / Radio I...
1299,1299,Single AGNs,-99,Radio Imaging / Double Radio Sources,Radio Imaging,222051.44+005815.0,0.318100,-99,22:20:51.44,+00:58:15.0,Optical,SDSS,J2000,-100.0,-100.0,-100.0,222051.66+005815.8,0.317800,-99,22:20:51.66,+00:58:15.8,J2000,Optical,SDSS,-100.0,-100.0,-100.0,63.500000,2.600000,12.054830,-99.0,Fu+2015 ; Fu+2015b,2015ApJ...799...72F ; 2015ApJ...815L...6F,https://doi.org/10.1088/0004-637X/799/1/72 ; h...,Faranoff-Riley II radio galaxies located withi...,-1.0,Dual AGN Candidate,Single AGNs,Dual AGN Candidate,-99,-99,-99,-99,-99,68.060244,Radio Imaging / Double Radio Sources / Radio I...
1799,1799,Binary Quasar Candidate,-99,Fiber Optical Spectroscopy / Optical Spectrosc...,-99,SDSS J0818+0601A,2.359000,spec,08:18:30.46,+06:01:38.0,Optical,SDSS,J2000,-100.0,-100,-100,SDSS J0818+0601B,2.361000,spec,08:18:30.420,+06:01:37.860,J2000,Optical,SDSS,-100.0,-100,-100,-177.976175,1.100000,8.984129,-99.0,More+2016 ; Hutsemekers+2020,2016MNRAS.456.1595M ; 2020A&A...633A.101H,https://doi.org/10.1093/mnras/stv2813 ; https:...,Hutsemekers+ find in their spectropolarimetric...,-1.0,Dual AGN Candidate,Binary Quasar Candidate,Dual AGN Candidate,-99,-99,-99,-99,-99,-177.976175,Fiber Optical Spectroscopy / Optical Spectrosc...
2143,2143,Dual SMBH Candidate,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,Radio Imaging,J101847.57+294114.1,0.082210,spec,10:18:47.57,+29:41:14.1,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,-51.300000,3.000000,-99.000000,-99.0,Comerford+2014 ; Müller-Sánchez,2014ApJ...789..112C ; 2016ApJ...830...50M,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Muller-Sanchez+2016 used OSIRIS IFS observati...,-1.0,Dual SMBH Candidate,Dual SMBH Candidate,Dual SMBH Candidate,-99,-99,-99,-99,-99,-99.000000,Optical Spectroscopy / Fiber Spectroscopy / Ve...
2165,2165,Dual SMBH Candidate,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,Radio Imaging,J105553.64+152027.5,0.092005,spec,10:55:53.64,+15:20:27.5,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,-112.600000,3.000000,-99.000000,-99.0,Comerford+2014 ; Müller-Sánchez,2014ApJ...789..112C ; 2016ApJ...830...50M,https://doi.org/10.1088/0004-637X/789/2/112 ; ...,Muller-Sanchez+2016 used OSIRIS IFS observati...,-1.0,Dual SMBH Candidate,Dual SMBH Candidate,Dual SMBH Candidate,-99,-99,-99,-99,-99,-99.000000,Optical Spectroscopy / Fiber Spectroscopy / Ve...
2176,2176,Dual SMBH Candidate,-99,Optical Spectroscopy / Fiber Spectroscopy / Ve...,Radio Imaging,J111729.22+614015.2,0.111929,spec,11:17:29.22,+61:40:15.2,Optical,SDSS,J2000,-100.0,-100,-100,-99,-99.000000,-99,-99,-99,-99,-99,-99,-100.0,-100,-100,84.900000,3.000000,-99.000000,-99.0,Comerford+2014 ; Müller-Sánchez,2014ApJ...789..112C ; 2016ApJ...830...50M,https://doi.org/10.10

In [560]:
#MAC[MAC['Confirmation Method']!='-99']

In [ ]:
gloin = gimli[(gimli['z1']>0) & (gimli['z2']<0)]
oin = gimli[(gimli['z1']>0) & (gimli['z2']>0)]
bombur = gimli[(gimli['z1']<0) & (gimli['z2']<0)]



In [ ]:
kili = gimli[(gimli['RA1']!=gimli['RA2']) & (gimli['RA2']!=-99)]

print(len(kili))
kili

In [ ]:
fili = gimli[gimli['Name2']!='-99']

print(len(fili))
fili

In [ ]:
gloin

In [ ]:
oin

In [ ]:
bombur

In [ ]:
#for index, row in eftek2017.iterrows():
#    if (row['z1']!=-99) & (row['z2']!=-99):
#        eftek2017.at[index, 'dV'] = (2.99e+5)*((1+eftek2017.at[index,'z1'])**2 - (1+eftek2017.at[index,'z2'])**2)/((1+eftek2017.at[index,'z1'])**2+(1+eftek2017.at[index,'z2'])**2)
#    else:
#        eftek2017.at[index, 'dV'] = -99
        

In [ ]:
#unique_types = MAC['System Type'].dropna().unique()
#
#print("Unique System Types and combos:", unique_types)
#print(len(unique_types))


In [ ]:
## this was for finding the number of objects that did not have selection methods mentioned
## we no longer needed this to be an active cell now that we've fixed those issues
#gandalf_nosel = gandalf[gandalf['Selection Method']=='-99']
#
#gandalf['Selection Method'] = gandalf['Selection Method'].astype(str)
#len(gandalf_nosel)
#
#num = 0
#for index, row in gandalf.iterrows():
#    if '-99' in row['Selection Method']:
#        num += 1
#    else:
#        num += 0
#print(num)
#
##for index, row in gandalf.iterrows():
##    if '-99' in row['Selection Method']:
##        print(row)
#

In [ ]:
# saving the formatted version here
MAC = pd.concat([gandalf,gimli])
MAC.reset_index(drop=True, inplace=True)

MAC.to_csv('MAC_DR0p8_formatted.csv', sep=',', index=False)

In [ ]:
# now we'll work to format the selection methodology cell

types = MAC['Selection Method'].dropna().str.split(' / ')

# Step 2 and 3: Remove duplicates and alphabetize for each cell
def process_cell(cell):
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

# Step 4: Join the contents back into a single string
MAC['Processed Selection Method'] = processed_types.apply(' / '.join)

# Display the updated DataFrame
#print(MAC[['Processed Selection Method']])

In [ ]:
methods = MAC['Processed Selection Method'].dropna().str.split(' / ')

# Flatten the lists and find unique labels
unique_methods = set(method for sublist in methods for method in sublist)

print("Unique selection methods:", unique_methods)
print(len(unique_methods))


In [ ]:
methods = ['Optical Spectroscopic Emission Line Ratios','X-Shaped Radio Source','Double Radio Sources',\
           'Fiber Optical Spectroscopy','LOS Radial Velocity Shifts','Near-IR Imaging','Slit Optical Spectroscopy',\
           'Quasi-Periodicity','Optical-Mid-IR Colors','X-ray Spectroscopy','Mid-IR Imaging',\
           'X-ray Quasi-Periodicity','BAT Selection','Narrow-Band Optical imaging','Near-IR Periodicity',\
           'Radio-Optical Offsets','Slit Near-IR Spectroscopy','Varstrometry','Radio-to-infrared Colors',\
           'IFU Optical Imaging','Astrometry','Modeling','Optical Photometry','Gamma-Ray Quasi-Periodicity',\
           'Radio Periodicity','Periodicity','Serendipitous','Gamma-Ray Periodicity','SMBH Peculiar Motion',\
           'IR Luminosity','Velocity Offset Narrow Optical Spectroscopic Emission Lines','Near-IR Spectroscopy',\
           'Hard X-ray Periodicity','Optical Colors','X-ray Imaging','Long-Slit Optical Spectroscopy',\
           'Optical Periodicity','Velocity Offset Broad Optical Spectroscopic Emission Lines',\
           'Mid-IR Spectroscopy','Optical Spectroscopy','Double-Peaked Optical Spectroscopic Emission Lines',\
           'UV Spectroscopy','Double Jet','Double-Peaked Broad Optical Spectroscopic Emission Lines',\
           'IFU Optical Spectroscopy','Optical Quasi-Periodicity','Near-IR Colors',\
           'Double-Peaked Narrow UV Spectroscopic Emission Lines','X-ray Periodicity','Submillimeter Imaging',\
           'UV Imaging','Positional Offset','Slitless Optical Spectroscopy','Jet Precession',\
           'Radio Quasi-Periodicity','Fiber Spectroscopy','Near-IR Quasi-Periodicity','Variability',\
           'Mid-IR Colors','Radio Imaging','Optical Imaging']
#['Velocity Offset','Asymmetric','Serendipitous','Optical-Mid-IR Colors','Myers+','Extreme Velocity Offsets',\
#'IFU Optical Imaging','Optical Photometry','UV Spectroscopy','X-ray Imaging',\
#'Velocity Offset Optical Spectroscopic Emission Lines','Radial Velocity Shifts','BAT Selection',\
#'Optical Spectroscopic Emission Line Ratios ; Integral Field Unit Optical Spectroscopy',\
#'Narrow-Band Optical imaging','Optical Colors','Optical Spectroscopic Emission Line Ratios',\
#'Double-Peaked Broad Optical Spectroscopic Emission Lines ; Temporal Velocity Shifts in Mg II',\
#'Slit Optical Spectroscopy','Radio Imaging','Modeling','Jet precession','Lens','Jet Precession',\
#'Long-Slit Optical Spectroscopy','Radial Velocity Shifts ; Offset Broad Lines','Periodicity',\
#'Quasi-Periodicity','Submillimeter Imaging','Fiber Spectroscopy','X-Shaped Radio Source',\
#'Optical Light Curve Variability','SMBH Peculiar Motion','Mid-IR Spectroscopy','Complex. Needs review.',\
#'Optical Spectroscopy','Velocity Shifts between Narrow and Broad Lines','Velocity Offset Emission Lines',\
#'IR Colors','Radio-Optical Offsets','Integral Field Unit Optical Spectroscopy','Offset Broad Lines',\
#'Near-IR Imaging','X-ray Spectroscopy','Near-IR Slit Spectroscopy','Near-Infared Imaging',\
#'Radio Morphology+Helical Model or Structure','Mid-IR Colors','Fiber Spectra','Offset Emission Lines',\
#'1.487','Optical Flare','-99','Hard X-ray Imaging','Slitless Optical Spectroscopy',\
#'Possible Double cores+jets','Double Radio Sources','Double-Peaked Optical Spectroscopic Emission Lines',\
#'Low-Dispersion Spectroscopy','Optical Slit Spectroscopy','IFU Spectroscopy','Jet kinematics',\
#'Photocenter variability','Offset Optical Emission Lines','Optical Imaging','Positional Offsets',\
#'Optical Periodicity','Radial Velocity Shift','Spectroscopic Modeling','Mid-IR Imaging',\
#'Double-Peaked Spectroscopic Emission Lines','Hard X-ray Spectroscopy',\
#'Offset Optical Spectroscopic Emission Lines','Offset Broad Optical Lines',\
#'SED Features (Blue Excess)','Mid-infrared Selection','Time Varying Offset Spectroscopic Emission Lines',\
#'Near-IR Colors','Offset Spectroscopic Emission Lines','DDRG Modeling','Velocity Shifted Emission Lines',\
#'UV Imaging','Radio-to-infrared Colors','Variability','Near-IR Spectroscopy',\
#'Temporal Velocity Shifts in Mg II','Fiber Optical Spectroscopy','IFU Optical Spectroscopy',\
#'Varstrometry','Astrometry','Double-Peaked Broad Optical Spectroscopic Emission Lines',\
#'Double Jet','Positional Offset']

for i in methods:
    num = 0
    for index,row in MAC.iterrows():
        if i in row['Processed Selection Method']:
            num += 1
        else:
            num += 0
    print("Method "+str(i)+" identified "+str(num)+" times!")

# Adjust the selection technique for:
#E1821+643

# Note: we're shifting asymmetric to Velocity Offset Broad Optical Spectroscopic Emission Lines

# LQAC_052-000_030 --> check coords but should be optical imaging / optical spectroscopy / fiber spectroscopy
# LQAC_171+003_009 same
# LQAC_122+031_018 same 
# LQAC_179+028_004 same
# LQAC_230+056_004 same
# LQAC_250+039_019,hewitt and burbidge and then sdss for second
# LQAC_144+033_040 sdss and sdss
# LQAC_136+000_002 sdss and sdss
# LQAC_143+033_020 sdss and sdss
# SDSSJ165502.02+260516.5 / LQAC_253+026_011_012 sdss and sdss
# SDSSJ155218.09+045635.2 / LQAC_238+004_003_004 sdss and sdss

# LQAC_052-000_030 --> flagged now as bad; I've seen the optical imaging. 
# The fiber spectra is clearly contaminated and not centered on a companion. Flagged as -1.
# There is a weak optical companion but it is likely a star or a background quasar.
# LQAC_171+003_009--> I am suspicious of this as well but will leave it for now. Will change it's flag to 0

# all of these need to have their selection methods to changed to 'Optical Spectroscopy' and 'fiber spectroscopy' 



In [ ]:
# Now down below here we're going to start working on formatting the separations and ensuring we have \
# angular separations where needed/applicable and that we don't overwrite physical separations by accident


gg = gandalf_white[(gandalf_white['Sep(kpc)']>0) & (gandalf_white['Sep']<0)]
len(gg)
#gg

In [ ]:
gg = gandalf_grey[(gandalf_grey['Sep(kpc)']>0) & (gandalf_grey['Sep']<0)]
len(gg)
gg

In [ ]:
gg = gandalf_grey[(gandalf_grey['Sep']<0)]
len(gg)
#gg

In [ ]:
gg = gimli[(gimli['Sep(kpc)']>0) & (gimli['Sep']<0)]
len(gg)
gg

In [ ]:
types = MAC['System Type'].dropna().str.split(' / ')

# Flatten the lists and find unique labels
unique_types = set(method for sublist in types for method in sublist)

print("Unique System Types and combos:", unique_types)
print(len(unique_types))



In [ ]:
unique_combinations = MAC['Processed System Type'].dropna().unique()

print(unique_combinations)
print(len(unique_combinations))


In [ ]:
#for i,j in MAC.iterrows():
#    if MAC.at[i, 'Processed System Type']=='Quasar Pairs':
#        print(j)

In [ ]:
# And here we're going to introduce the subjective flag for system confidence

# literature and Name 1 == HVGC-1 is highly unlikely to be a recoil
# Lens Candidate,WISE 0326-3122 is a lens candidate also based on Schechter+2017
# Lens Candidate,WISE 1051-1142 is a lens candidate also based on Schechter+2017

# we flagged IRAS 16474+3430 as a dual agn candidate but it might be a real dual!

# (Unlikely),PKS 0537-441 listed oth a binary uqasar and as a smbh binary candidate but i consider it to be a very \
# weak binary candidate

# / Lens,Q1208+1011 could still be a lens based on literature. Need to double check

# cdouble check QJ0240-343AB

# double check Q0101.8-3012

# double check FBQ 1633+3134


# PG 1553+113 angular separation is an upper limit on the separation (<)

# for 4C+22.25 there was a binary separation (kpc) limit range of lim0.04-0.08e-3

# for Mrk 231 there was an angular limit of lim1e-3

# Tsai+2013's target was listed as having a 1pc separation....

# the Zhou+2004 ; Jaiswal+2019 target evidently had a matxch with another catalog and I'd previously listed multiple\
# separations.



In [ ]:
from astropy.table import Table


MAC1 = MAC.drop(['Notes'], axis=1)
t = Table.from_pandas(MAC1)

t.write('MAC_DR0p5_beta_webtable_16Jan2024.html', format='jsviewer', overwrite=True)



In [ ]:
# and now adjusting the selection, analysis, and confirmation methodologies....

#Comerford+2009a

#Mrk 273 asked imanishi, but still need to ask vivian u

will need to go through and check all binary BL papers


#Need to measure separations

#Comerford+2011 ask julie if she has distinct coordinates

#IRAS 16474+3430 M email imanishi and see if they have coordinates and separation

#,McGurk+2011 check other papers for distinct coordinates
#Just emailed McGurk about 2011 and 2015 paper

 Need to email Fu about 2012 and 2011 papers
# calculating here separations




In [ ]:
# here we are further formatting the coordinates columns
# we'll be flagging RA2 and Dec2 as -99 (value not string) for cases where RA2 and Dec2 are duplicated from RA1 and Dec1

# we'll start by flagging groups of objects based on the paper they come from:
papers = ['Orosz+2013','Hwang+2020']

for i in papers:
    for index, row in MAC[MAC['Paper(s)']==str(i)].iterrows():
        MAC.at[index, 'RA2'] = -99
        MAC.at[index, 'Dec2'] = -99

# and here we are formatting for the naming convention for Name2
# if the entry for Name 2 is duplicated from Name1, it is reflagged as -99. Here I have gone through and manually \
# checked this

papers = ['Kim+2020','Barrows+2013','Ge+2012','Lyu+2016 ; Yuan+2016','Lyu+2016','Yuan+2016','Wang+2009 ; Ge+2012',\
         'Liu+2010a ; Ge+2012','Ge+2012 ; Orosz+2013','Smith+2010 ; Song+2020','Shi+2014',\
         'Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020','Smith+2010 ; Kim+2020 ; Song+2020',\
         'Smith+2010 ; Smith+2012 ; Song+2020','Smith+2010 ; Ge+2012 ; Song+2020','Wang+2009 ; Tingay+2011 ; Ge+2012',\
         'Wang+2009 ; Liu+2010a ; Ge+2012','Liu+2010a ; Yuan+2016','Yuan+2016 ; Yang+2019 ; Joshi+2019',\
         'Kim+2020 ; Liu+2014','Liu+2010a ; Yuan+2016','Shi+2014 ; Lyu+2016','Smith+2010 ; Yuan+2016 ; Song+2020',\
         'Smith+2010 ; Shi+2014 ; Song+2020','Smith+2010 ; Comerford+2018 ; Song+2020',\
         'Liu+2010a ; Ge+2012 ; Yuan+2016','Liu+2010a ; Yuan+2016 ; Comerford+2018','Ge+2012 ; Yuan+2016',\
         'Wang+2009 ; Shi+2014','Liu+2010a ; Ge+2012 ; Comerford+2018','Liu+2010a','Ge+2012 ; Kim+2020']
#J143359.71+351020.5 (comerford2013)
for i in papers:
    for index, row in MAC[MAC['Paper(s)']==str(i)].iterrows():
        MAC.at[index, 'Name2'] = "-99"

# just added A and B designations to the Miller+2004 targets, since previously they were listed as the same thing

# and evidently the objects within the gimli table have already been taken care of, so we only need to be \
# concerned with the gandalf targets


In [ ]:
gandalfcheck = gandalf[(gandalf['RA1'].astype(str)==gandalf['RA2'].astype(str)) & (gandalf['Dec1'].astype(str)==gandalf['Dec2'].astype(str))]

gandalfcheck

In [ ]:
MACcheck = MAC[(MAC['Name1']==MAC['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
print(len(MACcheck))
MACcheck

In [ ]:
gandalfcheck = gandalf[(gandalf['Name1']==gandalf['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
print(len(gandalfcheck))
gandalfcheck

In [ ]:
gandalfcheck = gandalf[(gandalf['Name1']==gandalf['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
print(len(gandalfcheck))
gandalfcheck



In [ ]:
gimlicheck = gimli[(gimli['Name1']==gimli['Name2'])]#gandalf[(gandalf['RA2']=='-99')]
print(len(gimlicheck))
gimlicheck

# did this check; no objects that have matching Name1 and Name2

In [ ]:
gimli

In [ ]:
    #if row['Paper(s)'] == 'Hwang+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Orosz+2013':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Lyu+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Yuan+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Barrows+2013':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Ge+2012':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Lyu+2016 ; Yuan+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Wang+2009 ; Liu+2010a ; Ge+2012':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Song+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif row['Paper(s)'] == 'Shi+2014':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif row['Paper(s)'] == 'Wang+2009 ; Ge+2012 ; Shi+2014':
    #    gandalf.at[index, 'Confidence Flag'] = 0                    
    #elif row['Paper(s)'] == 'Liu+2010a ; Ge+2012':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif row['Paper(s)'] == 'Liu+2010a':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif row['Paper(s)'] == 'Kim+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0               
    #elif row['Paper(s)'] == 'Spiniello+2018':
    #    gandalf.at[index, 'Confidence Flag'] = 0        
    #elif row['Paper(s)'] == 'Rusu+2019':
    #    gandalf.at[index, 'Confidence Flag'] = 0 
    #elif row['Paper(s)'] == 'Wang+2009 ; Ge+2012':
    #    gandalf.at[index, 'Confidence Flag'] = 0 
    #elif row['Paper(s)'] == 'Wang+2009':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Yuan+2016 ; Cheung+2007 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Liu+2010a ; Yuan+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Liu+2010a ; Ge+2012 ; Yuan+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Ge+2012 ; Orosz+2013':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Wang+2009 ; Shi+2014':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Ge+2012 ; Comerford+2013':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Smith+2012 ; Song+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Ge+2012 ; Song+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Smith+2012 ; Ge+2012 ; Song+2020 ':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Smith+2012 ; Song+2020 ':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Smith+2010 ; Ge+2012 ; Song+2020 ':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Yuan+2016 ; Yang+2019 ; Joshi+2019':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Kim+2020 ; Liu+2014':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Kim+2020 ; Kim+2016':
    #    gandalf.at[index, 'Confidence Flag'] = 0
    #elif row['Paper(s)'] == 'Spiniello+2018 ; Rusu+2019':
    #    gandalf.at[index, 'Confidence Flag'] = 0
#shi+2014 --> 0
#Wang+2009 ; Ge+2012 ; Shi+2014
#Liu+2010a ; Ge+2012
#Liu+2010a
#Kim+2020
###Kim+2020 ; Song+2020 --> figure out why this exists without Smith+2010. it shouldn't I don't think
### Kim+2020 ; Foord+2020 same as above
#Findlay+2018
#Spiniello+2018 0 
#Rusu+2019 0 
##Inada+2012 0
##Inada+2008 0
##Inada+2010 0
#Lemon+2018 +0.5
#Lemon+2019 +0.5
#Lemon+2020 +0.5
# Koss 2012 +1
# Hennawi+2010 +1
# Hennawi+2006 +1
# Hennawi+2006 ; Eftekharzadeh+2017 +1
# Hennawi+2006 ; Inada+2008 ; Eftekharzadeh+2017 +1
# Hennawi+2006 ; Inada+2008 ; Lemon+2018 +1
#Liu+2011b ; Ge+2012
#Liu+2010a ; Yuan+2016
#Liu+2010a ; Ge+2012 ; Yuan+2016
#Ge+2012 ; Orosz+2013
#Wang+2009 ; Shi+2014
#Ge+2012 ; Comerford+2013
#Comerford+2013
#Smith+2010 ; Smith+2012 ; Song+2020
#Smith+2010 ; Ge+2012 ; Song+2020
#Yuan+2016 ; Yang+2019 ; Joshi+2019
#Kim+2020 ; Liu+2014
#Spiniello+2018 ; Rusu+2019
#Yuan+2016 ; Cheung+2007 ; Roberts+2018 ; Saripalli+2018 ; Saripalli+2018
#Grade A targets from Fu+2015. --> +0.5
#Grade B targets from Fu+2015. --> 0
#Fu+2018 --> +1

In [ ]:
#MAC1

In [ ]:
## here we're beginning to clean up the tables and remove projected pairs or pairs that are not close enough to \
## be 'merger-induced' (like clustered quasars)
#
## first we're checking to see if the coordinates and redshifts for AGN 1 and 2 are identical under some conditions.\
## If they are, we'll be replacing the values with a no-value flag. This is in preparation for clipping on \
## separations and velocity differences
#MAC.fillna(-99, inplace=True)
#
#MAC['z1'] = MAC['z1'].astype(float)
#MAC['z2'] = MAC['z2'].astype(float)
#
#
##for i,j in MAC.iterrows():
##    if ('Double-Peak' in MAC.at[i, 'Selection Method']) and (MAC.at[i, 'z1']==MAC.at[i, 'z2']):
##        print('True')
#
#for i, row in MAC.iterrows():
#    # Check if z1 and z2 are not -99
#    if row['z1'] != -99 and row['z2'] != -99:
#        try:
#            # Perform the calculation for the row
#            MAC.at[i, 'dV'] = (2.99e+5) * ((1 + row['z1'])**2 - (1 + row['z2'])**2) / ((1 + row['z1'])**2 + (1 + row['z2'])**2)
#        except TypeError as e:
#            # If a TypeError occurs, print the row index and the error
#            print(f"Error at row {i}: {e}")
#            print(row)
#
#            
#for i, row in MAC.iterrows():
#    # Check if z1 and z2 are not -99
#    if (row['Sep'] != -99) and (row['Sep'] != -99.0) and (row['z1']>0) and (row['Sep']>0):
#        try:
#            # Perform the calculation for the row
#            row['Sep(kpc)'] = row['Sep']*cosmo.kpc_proper_per_arcmin(row['z1'])*(u.arcmin/u.kpc)*(1/60)
#        except TypeError as e:
#            # If a TypeError occurs, print the row index and the error
#            print(f"Error at row {i}: {e}")
#            print(row)
#

In [ ]:
#MAC_bad1 = MAC[MAC['Sep(kpc)']>100]


In [ ]:
#for i, row in MAC.iterrows():
#    # Check if z1 and z2 are not -99
#    try:
#        # Perform the calculation for the row
#        MAC_bad1 = MAC[MAC['Sep(kpc)']>100.0]
#    except TypeError as e:
#        # If a TypeError occurs, print the row index and the error
#        print(f"Error at row {i}: {e}")
#        print(row)

In [ ]:
##import pandas as pd
#
## Load your dataframe
##df = pd.read_csv('your_file.csv')  # Replace with your file path
#
## Function to check if a value is a float
#def is_float(value):
#    try:
#        float(value)
#        return True
#    except ValueError:
#        return False
#
## Create a new dataframe for problem rows
#problem_rows = pd.DataFrame()
#
## Iterate over the rows
#for index, row in MAC.iterrows():
#    value = row['Sep(kpc)']  # Replace 'your_column' with the name of your column
#    if not is_float(value) or value == -99:
#        problem_rows = problem_rows.append(row)
#
## Now, 'problem_rows' contains all the rows that you need to examine
##print(problem_rows)
#
## Optionally, save the problem rows to a new CSV
##problem_rows.to_csv('problem_rows.csv', index=False)
#

In [ ]:
#problem_rows

In [ ]:
##import pandas as pd
#
## Load your dataframe
##df = pd.read_csv('your_file.csv')  # Replace with your file path
#
## Function to check if a value is a float
#def is_float(value):
#    try:
#        float(value)
#        return True
#    except ValueError:
#        return False
#
## Create a new dataframe for problem rows
#good_rows = pd.DataFrame()
#
## Iterate over the rows
#for index, row in MAC.iterrows():
#    value = row['Sep(kpc)']  # Replace 'your_column' with the name of your column
#    if is_float(value) or value != -99 or value!=-99.0:
#        good_rows = good_rows.append(row)
#
## Now, 'problem_rows' contains all the rows that you need to examine
##print(good_rows)
#
## Optionally, save the problem rows to a new CSV
##problem_rows.to_csv('problem_rows.csv', index=False)
#
#

In [ ]:
#MAC['Sep(kpc)'] = MAC['Sep(kpc)'].astype(float)


In [ ]:
#np.max(MAC['Sep(kpc)'])
#MAC

In [ ]:
#MAC_bad1 = MAC[MAC['Sep(kpc)']>100.0]




In [ ]:
#MAC_bad1

In [ ]:
#len(good_rows)

In [ ]:
#MAC_bad1 = good_rows[good_rows['Sep(kpc)']>100.0]



In [ ]:
## Function to check if a value in a column is a string
#def is_string(value):
#    return isinstance(value, str)
#
## Create a new dataframe for rows where the specified column contains a string
#rows_with_string_in_column = pd.DataFrame()
#
## Name of the column to check
#column_name = 'Sep(kpc)'  # Replace 'your_column' with the name of your column
#
## Iterate over the rows
#for index, row in MAC.iterrows():
#    if is_string(row[column_name]):
#        rows_with_string_in_column = rows_with_string_in_column.append(row)
#
## 'rows_with_string_in_column' contains all the rows where the specified column has a string
#print(rows_with_string_in_column)
#
## Optionally, save these rows to a new CSV
##rows_with_string_in_column.to_csv('rows_with_string_in_column.csv', index=False)
#

In [ ]:
#len(problem_rows)


In [ ]:
#problem_rows

In [ ]:
#for index, row in MAC.iterrows():
#    print(row['z1'].dtype,row['z2'].dtype)
    

In [ ]:
#MAC

In [ ]:
#cosmo.kpc_proper_per_arcmin(3)*(u.arcmin/u.kpc)*(1/60)

In [ ]:
## now we'll work to format the selection methodology cell
#
#types = MAC['Selection Method'].dropna().str.split(' / ')
#
## Step 2 and 3: Remove duplicates and alphabetize for each cell
#def process_cell(cell):
#    # Remove duplicates using set and then convert back to list
#    unique_labels = list(set(cell))
#    # Alphabetize the contents
#    unique_labels.sort()
#    return unique_labels
#
#processed_types = types.apply(process_cell)
#
## Step 4: Join the contents back into a single string
#MAC['Processed Selection Method'] = processed_types.apply(' / '.join)
#
## Display the updated DataFrame
##print(MAC[['Processed Selection Method']])

In [ ]:
#bad10 = MAC[MAC['Processed Selection Method']=='1.487']
#bad10